In [105]:
import numpy as np
import pandas as pd
from google.cloud import bigquery
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.express as px
import db_dtypes
import bigframes.pandas as bpd
from IPython.display import display, HTML
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import math
import statsmodels.api as sm
from datetime import datetime
from typing import Union
import logging
import sys

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s:%(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)


# growth_data = """

# WITH holdout_entities AS (
#   SELECT
#         entity_id
#        ,`Release Date` as release_date
#   FROM `logistics-data-storage-staging.long_term_pricing.global_holdout_rollout`
#   WHERE `Release Date` < DATE_SUB(CURRENT_DATE(), INTERVAL EXTRACT(DAYOFWEEK FROM CURRENT_DATE())- 2 DAY)
# ),
# customer_information AS (
#   SELECT
#          e.entity_id
#         ,e.release_date
#         ,IF(d.created_date <= e.release_date, FALSE, COALESCE(is_customer_holdout, FALSE)) AS is_customer_holdout
#         ,customer_id
#   FROM `fulfillment-dwh-production.cl.dps_holdout_users` AS d
#   JOIN holdout_entities AS e
#         ON d.entity_id = e.entity_id
#   WHERE d.created_date >= date('2025-01-01')
#     AND created_date < CURRENT_DATE()  -- data is not full/not present for current date
#     AND customer_id IS NOT NULL
#     AND customer_id NOT IN UNNEST(ARRAY(SELECT id FROM `fulfillment-dwh-production.cl._bad_dps_logs_ids`)) -- get rid of ids like 'null', 'test' etc
#     --AND d.entity_id in ('FP_MM','GV_CI', 'PY_UY','GV_BA','PY_NI','TB_IQ','TB_OM','PY_HN','TB_KW','GV_ME','PO_FI','FP_SG','FP_TH','HS_SA','FY_CY','TB_JO','GV_RS','GV_UG','GV_NG','NP_HU')
#   GROUP BY 1, 2, 3, 4
# )
#   SELECT 
#      e.entity_id
#     ,e.customer_id customer_id
#     ,e.is_customer_holdout
#     ,COUNT(DISTINCT case when dps.created_date <= e.release_date then dps.platform_order_code end) AS orders_pre
#     ,COUNT(DISTINCT case when dps.created_date > e.release_date then dps.platform_order_code end) AS orders_post
#     ,SUM(case when dps.created_date <= e.release_date then dps.fully_loaded_gross_profit_eur end) AS flgp_pre
#     ,SUM(case when dps.created_date > e.release_date then dps.fully_loaded_gross_profit_eur end) AS flgp_post
#   FROM customer_information e
#   LEFT JOIN `fulfillment-dwh-production.cl.dps_sessions_mapped_to_orders` dps
#     ON dps.entity_id = e.entity_id
#     AND dps.dps_customer_id = e.customer_id
#   WHERE created_date >= DATE_SUB(release_date, INTERVAL 8 WEEK)
#   AND created_date < CURRENT_DATE
#   AND platform_order_code IS NOT NULL
#   AND is_own_delivery
#   AND is_sent
#   --AND dps.entity_id in ('FP_MM','GV_CI', 'PY_UY','GV_BA','PY_NI','TB_IQ','TB_OM','PY_HN','TB_KW','GV_ME','PO_FI','FP_SG','FP_TH','HS_SA','FY_CY','TB_JO','GV_RS','GV_UG','GV_NG','NP_HU')
#   AND vendor_vertical_parent in ('Restaurant','restaurant','restaurants')
#   GROUP BY 1,2,3

#   """


# mkt_data = """
  
#   WITH holdout_entities AS (
#   SELECT
#         entity_id
#        ,`Release Date` as release_date
#   FROM `logistics-data-storage-staging.long_term_pricing.global_holdout_rollout`
#   WHERE `Release Date` < DATE_SUB(CURRENT_DATE(), INTERVAL EXTRACT(DAYOFWEEK FROM CURRENT_DATE())- 2 DAY)
# ),
# customer_information AS (
#   SELECT
#          e.entity_id
#         ,e.release_date
#         ,IF(d.created_date <= e.release_date, FALSE, COALESCE(is_customer_holdout, FALSE)) AS is_customer_holdout
#         ,customer_id
#   FROM `fulfillment-dwh-production.cl.dps_holdout_users` AS d
#   JOIN holdout_entities AS e
#         ON d.entity_id = e.entity_id
#   WHERE d.created_date >= date('2025-01-01')
#     AND created_date < CURRENT_DATE()  -- data is not full/not present for current date
#     AND customer_id IS NOT NULL
#     AND customer_id NOT IN UNNEST(ARRAY(SELECT id FROM `fulfillment-dwh-production.cl._bad_dps_logs_ids`)) -- get rid of ids like 'null', 'test' etc
#   GROUP BY 1, 2, 3, 4
# )
#   SELECT 
#      e.entity_id
#     ,e.customer_id customer_id
#     ,e.is_customer_holdout
#     ,COUNT(DISTINCT case when mkt.order_date <= e.release_date then mkt.order_id end) AS orders_pre
#     ,COUNT(DISTINCT case when mkt.order_date > e.release_date then mkt.order_id end) AS orders_post
#     ,SUM(case when mkt.order_date <= e.release_date then mkt.analytical_profit end) AS analytical_profit_pre
#     ,SUM(case when mkt.order_date > e.release_date then mkt.analytical_profit end) AS analytical_profit_post
#     ,SUM(case when mkt.order_date <= e.release_date then mkt.local_analytical_profit end) AS local_analytical_profit_pre
#     ,SUM(case when mkt.order_date > e.release_date then mkt.local_analytical_profit end) AS local_analytical_profit_post
#     ,SUM(case when mkt.order_date <= e.release_date then mkt.revenue_net end) AS revenue_pre
#     ,SUM(case when mkt.order_date > e.release_date then mkt.revenue_net end) AS revenue_post
#     ,SUM(case when mkt.order_date <= e.release_date then mkt.cost_of_sales end) AS cost_of_sales_pre
#     ,SUM(case when mkt.order_date > e.release_date then mkt.cost_of_sales end) AS cost_of_sales_post
#   FROM customer_information e
#   LEFT JOIN `fulfillment-dwh-production.cl.dps_sessions_mapped_to_orders` dps
#     ON dps.entity_id = e.entity_id
#     AND dps.dps_customer_id = e.customer_id 
#   LEFT JOIN fulfillment-dwh-production.curated_data_shared_mkt.bima_order_profitability mkt
#     ON mkt.global_entity_id = dps.entity_id
#     AND mkt.order_id = dps.platform_order_code
#   WHERE order_date >= DATE_SUB(release_date, INTERVAL 8 WEEK)
#   AND order_date < CURRENT_DATE
#   and created_date >= DATE_SUB(release_date, INTERVAL 8 WEEK)
#   and created_date < CURRENT_DATE
#   AND platform_order_code IS NOT NULL
#   AND is_own_delivery
#   AND is_sent
#   AND e.entity_id is not null
#   AND vendor_vertical_parent in ('Restaurant','restaurant','restaurants')
#   GROUP BY 1,2,3
  
#   """

# mkt_data = """
  
# WITH holdout_entities AS (
#   SELECT
#         entity_id
#        ,`Release Date` as release_date
#   FROM `logistics-data-storage-staging.long_term_pricing.global_holdout_rollout`
#   WHERE `Release Date` < DATE_SUB(CURRENT_DATE(), INTERVAL EXTRACT(DAYOFWEEK FROM CURRENT_DATE())- 2 DAY)
# ),
# customer_information AS (
#   SELECT
#          e.entity_id
#         ,e.release_date
#         ,IF(d.created_date <= e.release_date, FALSE, COALESCE(is_customer_holdout, FALSE)) AS is_customer_holdout
#         ,customer_id
#   FROM `fulfillment-dwh-production.cl.dps_holdout_users` AS d
#   JOIN holdout_entities AS e
#         ON d.entity_id = e.entity_id
#   WHERE d.created_date >= date('2025-01-01')
#     AND created_date < CURRENT_DATE()  -- data is not full/not present for current date
#     AND customer_id IS NOT NULL
#     AND customer_id NOT IN UNNEST(ARRAY(SELECT id FROM `fulfillment-dwh-production.cl._bad_dps_logs_ids`)) -- get rid of ids like 'null', 'test' etc
#   GROUP BY 1, 2, 3, 4
# )
#   SELECT 
#      e.entity_id
#     ,e.customer_id customer_id
#     ,e.is_customer_holdout
#     ,COUNT(DISTINCT case when mkt.order_date <= e.release_date then mkt.order_id end) AS orders_pre
#     ,COUNT(DISTINCT case when mkt.order_date > e.release_date then mkt.order_id end) AS orders_post
#     ,SUM(case when mkt.order_date <= e.release_date then mkt.analytical_profit end) AS analytical_profit_pre
#     ,SUM(case when mkt.order_date > e.release_date then mkt.analytical_profit end) AS analytical_profit_post
#     ,SUM(case when mkt.order_date <= e.release_date then mkt.local_analytical_profit end) AS local_analytical_profit_pre
#     ,SUM(case when mkt.order_date > e.release_date then mkt.local_analytical_profit end) AS local_analytical_profit_post
#     ,SUM(case when mkt.order_date <= e.release_date then mkt.revenue_net end) AS revenue_pre
#     ,SUM(case when mkt.order_date > e.release_date then mkt.revenue_net end) AS revenue_post
#     ,SUM(case when mkt.order_date <= e.release_date then mkt.cost_of_sales end) AS cost_of_sales_pre
#     ,SUM(case when mkt.order_date > e.release_date then mkt.cost_of_sales end) AS cost_of_sales_post
#     ,SUM(case when mkt.order_date <= e.release_date then mkt.dh_funded_basket_incentives end) AS dh_funded_basket_incentives_pre
#     ,SUM(case when mkt.order_date > e.release_date then mkt.dh_funded_basket_incentives end) AS dh_funded_basket_incentives_post
#   FROM customer_information e
#   LEFT JOIN `fulfillment-dwh-production.cl.dps_sessions_mapped_to_orders` dps
#     ON dps.entity_id = e.entity_id
#     AND dps.dps_customer_id = e.customer_id 
#   LEFT JOIN fulfillment-dwh-production.curated_data_shared_mkt.bima_order_profitability mkt
#     ON mkt.global_entity_id = dps.entity_id
#     AND mkt.order_id = dps.platform_order_code
#   WHERE order_date >= DATE_SUB(release_date, INTERVAL 8 WEEK)
#     AND order_date < CURRENT_DATE
#     and created_date >= DATE_SUB(release_date, INTERVAL 8 WEEK)
#     and created_date < CURRENT_DATE
#     AND platform_order_code IS NOT NULL
#     AND is_own_delivery
#     AND is_sent
#     AND e.entity_id is not null
#     AND vendor_vertical_parent in ('Restaurant','restaurant','restaurants')
#   GROUP BY 1,2,3
  
# """


# marketing_data = """
# WITH eligible_holdout_entities AS (
#   SELECT
#         entity_id
#        ,`Release Date` as release_date
#   FROM `logistics-data-storage-staging.long_term_pricing.global_holdout_rollout`
#   WHERE `Release Date` < DATE_SUB(CURRENT_DATE(), INTERVAL EXTRACT(DAYOFWEEK FROM CURRENT_DATE())- 2 DAY)
# ),
# customer_information AS (
#   SELECT
#          e.entity_id
#         ,e.release_date
#         ,COUNT(DISTINCT CASE WHEN is_customer_holdout IS True THEN customer_id END) AS holdout_customers
#         ,COUNT(DISTINCT CASE WHEN is_customer_holdout IS False THEN customer_id END) AS non_holdout_customers
#         ,COUNT(DISTINCT customer_id) AS all_customers
#   FROM `fulfillment-dwh-production.cl.dps_holdout_users` AS d
#   JOIN eligible_holdout_entities AS e
#   ON d.entity_id = e.entity_id
#   WHERE d.created_date >= DATE('2025-01-01')
#     AND customer_id NOT IN UNNEST(ARRAY(SELECT id FROM `fulfillment-dwh-production.cl._bad_dps_logs_ids`))
#     AND d.created_date > e.release_date
#     AND customer_id IS NOT NULL
#   GROUP BY e.entity_id, e.release_date
# ), 
# orders as (
# select
#       DATE_SUB(CURRENT_DATE(), INTERVAL EXTRACT(DAYOFWEEK FROM CURRENT_DATE()) - 2 DAY) AS analysis_week 
#     ,global_entity_id
#     ,release_date
#     ,holdout_customers
#     ,non_holdout_customers
#     ,all_customers
#     ,count(distinct case when dps.is_customer_holdout then op.order_id end) holdout_orders
#     ,count(distinct case when dps.is_customer_holdout is false then op.order_id end) non_holdout_orders
#     ,sum(case when dps.is_customer_holdout then op.gmv end) holdout_gmv
#     ,sum(case when dps.is_customer_holdout is false then op.gmv end) non_holdout_gmv
#     ,sum(case when dps.is_customer_holdout then op.gfv end) holdout_gfv
#     ,sum(case when dps.is_customer_holdout is false then op.gfv end) non_holdout_gfv
#     ,sum(case when dps.is_customer_holdout then op.analytical_profit end) flgp_holdout
#     ,sum(case when dps.is_customer_holdout is false then op.analytical_profit end) flgp_non_holdout
#     ,sum(case when dps.is_customer_holdout then op.revenue_net end) holdout_revenue_net
#     ,sum(case when dps.is_customer_holdout is false then op.revenue_net end) non_holdout_revenue_net
#     ,sum(case when dps.is_customer_holdout then op.delivery_costs end) holdout_delivery_costs
#     ,sum(case when dps.is_customer_holdout is false then op.delivery_costs end) non_holdout_delivery_costs
#     ,sum(case when dps.is_customer_holdout then op.delivery_fee_net end) holdout_delivery_fee_net
#     ,sum(case when dps.is_customer_holdout is false then op.delivery_fee_net end) non_holdout_delivery_fee_net
#     ,sum(case when dps.is_customer_holdout then dh_funded_basket_incentives end) holdout_dh_funded_basket_incentives
#     ,sum(case when dps.is_customer_holdout is false then dh_funded_basket_incentives end) non_holdout_dh_funded_basket_incentives
#     ,sum(case when dps.is_customer_holdout then cost_of_sales end) holdout_cost_of_sales
#     ,sum(case when dps.is_customer_holdout is false then cost_of_sales end) non_holdout_cost_of_sales
#     ,sum(case when dps.is_customer_holdout then cost_of_goods_sold end) holdout_cost_of_goods_sold
#     ,sum(case when dps.is_customer_holdout is false then cost_of_goods_sold end) non_holdout_cost_of_goods_sold
#     ,sum(case when dps.is_customer_holdout then payment_costs end) holdout_payment_costs
#     ,sum(case when dps.is_customer_holdout is false then payment_costs end) non_holdout_payment_costs
#     ,sum(case when dps.is_customer_holdout then recooks_and_wastage end) holdout_recooks_and_wastage
#     ,sum(case when dps.is_customer_holdout is false then recooks_and_wastage end) non_holdout_recooks_and_wastage
#     ,sum(case when dps.is_customer_holdout then refunds_and_compensation end) holdout_refunds_and_compensation
#     ,sum(case when dps.is_customer_holdout is false then refunds_and_compensation end) non_holdout_refunds_and_compensation
#     ,sum(case when dps.is_customer_holdout then semi_variable_delivery_costs end) holdout_semi_variable_delivery_costs
#     ,sum(case when dps.is_customer_holdout is false then semi_variable_delivery_costs end) non_holdout_semi_variable_delivery_costs
#     ,sum(case when dps.is_customer_holdout then contact_center_costs end) holdout_contact_center_costs
#     ,sum(case when dps.is_customer_holdout is false then contact_center_costs end) non_holdout_contact_center_costs
#     ,sum(case when dps.is_customer_holdout then other_non_specific_delivery_costs end) holdout_other_non_specific_delivery_costs
#     ,sum(case when dps.is_customer_holdout is false then other_non_specific_delivery_costs end) non_holdout_other_non_specific_delivery_costs
    
#     ,sum(case when dps.is_customer_holdout then commission_fee_net end) holdout_commission_fee_net
#     ,sum(case when dps.is_customer_holdout is false then commission_fee_net end) non_holdout_commission_fee_net

#     ,sum(case when dps.is_customer_holdout then sale_of_goods_revenue_net end) holdout_sale_of_goods_revenue_net
#     ,sum(case when dps.is_customer_holdout is false then sale_of_goods_revenue_net end) non_holdout_sale_of_goods_revenue_net
    
#     ,sum(case when dps.is_customer_holdout then card_fee_net end) holdout_card_fee_net
#     ,sum(case when dps.is_customer_holdout is false then card_fee_net end) non_holdout_card_fee_net

#     ,sum(case when dps.is_customer_holdout then priority_fee_net end) holdout_priority_fee_net
#     ,sum(case when dps.is_customer_holdout is false then priority_fee_net end) non_holdout_priority_fee_net

#     ,sum(case when dps.is_customer_holdout then bad_weather_fee_net end) holdout_bad_weather_fee_net
#     ,sum(case when dps.is_customer_holdout is false then bad_weather_fee_net end) non_holdout_bad_weather_fee_net

#     ,sum(case when dps.is_customer_holdout then vendor_funded_delivery_fee_subsidy_net end) holdout_vendor_funded_delivery_fee_subsidy_net
#     ,sum(case when dps.is_customer_holdout is false then vendor_funded_delivery_fee_subsidy_net end) non_holdout_vendor_funded_delivery_fee_subsidy_net

#     ,sum(case when dps.is_customer_holdout then service_fee_net end) holdout_service_fee_net
#     ,sum(case when dps.is_customer_holdout is false then service_fee_net end) non_holdout_service_fee_net

#     ,sum(case when dps.is_customer_holdout then listing_fee_net end) holdout_listing_fee_net
#     ,sum(case when dps.is_customer_holdout is false then listing_fee_net end) non_holdout_listing_fee_net

#     ,sum(case when dps.is_customer_holdout then sublease_revenue_net end) holdout_sublease_revenue_net
#     ,sum(case when dps.is_customer_holdout is false then sublease_revenue_net end) non_holdout_sublease_revenue_net

#     ,sum(case when dps.is_customer_holdout then ads_revenue_net end) holdout_ads_revenue_net
#     ,sum(case when dps.is_customer_holdout is false then ads_revenue_net end) non_holdout_ads_revenue_net

#     ,sum(case when dps.is_customer_holdout then joker_vendor_fee_net end) holdout_joker_vendor_fee_net
#     ,sum(case when dps.is_customer_holdout is false then joker_vendor_fee_net end) non_holdout_joker_vendor_fee_net

#     ,sum(case when dps.is_customer_holdout then other_ncr_net end) holdout_other_ncr_net
#     ,sum(case when dps.is_customer_holdout is false then other_ncr_net end) non_holdout_other_ncr_net

#     ,sum(case when dps.is_customer_holdout then small_basket_fee_net end) holdout_small_basket_fee_net
#     ,sum(case when dps.is_customer_holdout is false then small_basket_fee_net end) non_holdout_small_basket_fee_net

#     ,sum(case when dps.is_customer_holdout then subscription_fee_net end) holdout_subscription_fee_net
#     ,sum(case when dps.is_customer_holdout is false then subscription_fee_net end) non_holdout_subscription_fee_net

#     ,sum(case when dps.is_customer_holdout then subscription_vendor_subsidy_net end) holdout_subscription_vendor_subsidy_net
#     ,sum(case when dps.is_customer_holdout is false then subscription_vendor_subsidy_net end) non_holdout_subscription_vendor_subsidy_net

#     ,sum(case when dps.is_customer_holdout then other_revenue_net end) holdout_other_revenue_net
#     ,sum(case when dps.is_customer_holdout is false then other_revenue_net end) non_holdout_other_revenue_net

# from fulfillment-dwh-production.curated_data_shared_mkt.bima_order_profitability op
# JOIN customer_information usi 
#     ON usi.entity_id = op.global_entity_id
# JOIN `fulfillment-dwh-production.cl.dps_sessions_mapped_to_orders` dps 
#     ON dps.entity_id = op.global_entity_id
#     and dps.platform_order_code = op.order_id
#     and dps.created_date >= '2025-01-01'
#     and dps.created_date > usi.release_date
# WHERE op.order_date > usi.release_date
#     AND op.order_date >= '2025-01-01'
#     AND dps.is_own_delivery
#     AND dps.is_sent 
#     AND vendor_vertical_parent in ('Restaurant','restaurant','restaurants')
# GROUP BY 1,2,3,4,5,6
# )
# select * 
# from orders

# """


# marketing_data = """
# WITH eligible_holdout_entities AS (
#   SELECT
#         entity_id
#        ,`Release Date` as release_date
#   FROM `logistics-data-storage-staging.long_term_pricing.global_holdout_rollout`
#   WHERE `Release Date` < DATE_SUB(CURRENT_DATE(), INTERVAL EXTRACT(DAYOFWEEK FROM CURRENT_DATE())- 2 DAY)
# ),
# customer_information AS (
#   SELECT
#          e.entity_id
#         ,e.release_date
#         ,COUNT(DISTINCT CASE WHEN is_customer_holdout IS True THEN customer_id END) AS holdout_customers
#         ,COUNT(DISTINCT CASE WHEN is_customer_holdout IS False THEN customer_id END) AS non_holdout_customers
#         ,COUNT(DISTINCT customer_id) AS all_customers
#   FROM `fulfillment-dwh-production.cl.dps_holdout_users` AS d
#   JOIN eligible_holdout_entities AS e
#   ON d.entity_id = e.entity_id
#   WHERE d.created_date >= DATE('2025-01-01')
#     AND customer_id NOT IN UNNEST(ARRAY(SELECT id FROM `fulfillment-dwh-production.cl._bad_dps_logs_ids`))
#     AND d.created_date > e.release_date
#     AND customer_id IS NOT NULL
#   GROUP BY e.entity_id, e.release_date
# ), 
# orders as (
# select
#       DATE_SUB(CURRENT_DATE(), INTERVAL EXTRACT(DAYOFWEEK FROM CURRENT_DATE()) - 2 DAY) AS analysis_week 
#     ,global_entity_id
#     ,release_date
#     ,holdout_customers
#     ,non_holdout_customers
#     ,all_customers
#     ,count(distinct case when dps.is_customer_holdout then op.order_id end) holdout_orders
#     ,count(distinct case when dps.is_customer_holdout is false then op.order_id end) non_holdout_orders
#     ,sum(case when dps.is_customer_holdout then op.gmv end) holdout_gmv
#     ,sum(case when dps.is_customer_holdout is false then op.gmv end) non_holdout_gmv
#     ,sum(case when dps.is_customer_holdout then op.gfv end) holdout_gfv
#     ,sum(case when dps.is_customer_holdout is false then op.gfv end) non_holdout_gfv
#     ,sum(case when dps.is_customer_holdout then op.analytical_profit end) flgp_holdout
#     ,sum(case when dps.is_customer_holdout is false then op.analytical_profit end) flgp_non_holdout
#     ,sum(case when dps.is_customer_holdout then op.revenue_net end) holdout_revenue_net
#     ,sum(case when dps.is_customer_holdout is false then op.revenue_net end) non_holdout_revenue_net
#     ,sum(case when dps.is_customer_holdout then op.delivery_costs end) holdout_delivery_costs
#     ,sum(case when dps.is_customer_holdout is false then op.delivery_costs end) non_holdout_delivery_costs
#     ,sum(case when dps.is_customer_holdout then op.delivery_fee_net end) holdout_delivery_fee_net
#     ,sum(case when dps.is_customer_holdout is false then op.delivery_fee_net end) non_holdout_delivery_fee_net
#     ,sum(case when dps.is_customer_holdout then dh_funded_basket_incentives end) holdout_dh_funded_basket_incentives
#     ,sum(case when dps.is_customer_holdout is false then dh_funded_basket_incentives end) non_holdout_dh_funded_basket_incentives
#     ,sum(case when dps.is_customer_holdout then cost_of_sales end) holdout_cost_of_sales
#     ,sum(case when dps.is_customer_holdout is false then cost_of_sales end) non_holdout_cost_of_sales
#     ,sum(case when dps.is_customer_holdout then cost_of_goods_sold end) holdout_cost_of_goods_sold
#     ,sum(case when dps.is_customer_holdout is false then cost_of_goods_sold end) non_holdout_cost_of_goods_sold
#     ,sum(case when dps.is_customer_holdout then payment_costs end) holdout_payment_costs
#     ,sum(case when dps.is_customer_holdout is false then payment_costs end) non_holdout_payment_costs
#     ,sum(case when dps.is_customer_holdout then recooks_and_wastage end) holdout_recooks_and_wastage
#     ,sum(case when dps.is_customer_holdout is false then recooks_and_wastage end) non_holdout_recooks_and_wastage
#     ,sum(case when dps.is_customer_holdout then refunds_and_compensation end) holdout_refunds_and_compensation
#     ,sum(case when dps.is_customer_holdout is false then refunds_and_compensation end) non_holdout_refunds_and_compensation
#     ,sum(case when dps.is_customer_holdout then semi_variable_delivery_costs end) holdout_semi_variable_delivery_costs
#     ,sum(case when dps.is_customer_holdout is false then semi_variable_delivery_costs end) non_holdout_semi_variable_delivery_costs
#     ,sum(case when dps.is_customer_holdout then contact_center_costs end) holdout_contact_center_costs
#     ,sum(case when dps.is_customer_holdout is false then contact_center_costs end) non_holdout_contact_center_costs
#     ,sum(case when dps.is_customer_holdout then other_non_specific_delivery_costs end) holdout_other_non_specific_delivery_costs
#     ,sum(case when dps.is_customer_holdout is false then other_non_specific_delivery_costs end) non_holdout_other_non_specific_delivery_costs
    
#     ,sum(case when dps.is_customer_holdout then commission_fee_net end) holdout_commission_fee_net
#     ,sum(case when dps.is_customer_holdout is false then commission_fee_net end) non_holdout_commission_fee_net

#     ,sum(case when dps.is_customer_holdout then sale_of_goods_revenue_net end) holdout_sale_of_goods_revenue_net
#     ,sum(case when dps.is_customer_holdout is false then sale_of_goods_revenue_net end) non_holdout_sale_of_goods_revenue_net
    
#     ,sum(case when dps.is_customer_holdout then card_fee_net end) holdout_card_fee_net
#     ,sum(case when dps.is_customer_holdout is false then card_fee_net end) non_holdout_card_fee_net

#     ,sum(case when dps.is_customer_holdout then delivery_fee_net end) holdout_delivery_fee_net
#     ,sum(case when dps.is_customer_holdout is false then delivery_fee_net end) non_holdout_delivery_fee_net

#     ,sum(case when dps.is_customer_holdout then priority_fee_net end) holdout_priority_fee_net
#     ,sum(case when dps.is_customer_holdout is false then priority_fee_net end) non_holdout_priority_fee_net

#     ,sum(case when dps.is_customer_holdout then bad_weather_fee_net end) holdout_bad_weather_fee_net
#     ,sum(case when dps.is_customer_holdout is false then bad_weather_fee_net end) non_holdout_bad_weather_fee_net

#     ,sum(case when dps.is_customer_holdout then vendor_funded_delivery_fee_subsidy_net end) holdout_vendor_funded_delivery_fee_subsidy_net
#     ,sum(case when dps.is_customer_holdout is false then vendor_funded_delivery_fee_subsidy_net end) non_holdout_vendor_funded_delivery_fee_subsidy_net

#     ,sum(case when dps.is_customer_holdout then service_fee_net end) holdout_service_fee_net
#     ,sum(case when dps.is_customer_holdout is false then service_fee_net end) non_holdout_service_fee_net

#     ,sum(case when dps.is_customer_holdout then listing_fee_net end) holdout_listing_fee_net
#     ,sum(case when dps.is_customer_holdout is false then listing_fee_net end) non_holdout_listing_fee_net

#     ,sum(case when dps.is_customer_holdout then sublease_revenue_net end) holdout_sublease_revenue_net
#     ,sum(case when dps.is_customer_holdout is false then sublease_revenue_net end) non_holdout_sublease_revenue_net

#     ,sum(case when dps.is_customer_holdout then ads_revenue_net end) holdout_ads_revenue_net
#     ,sum(case when dps.is_customer_holdout is false then ads_revenue_net end) non_holdout_ads_revenue_net

#     ,sum(case when dps.is_customer_holdout then joker_vendor_fee_net end) holdout_joker_vendor_fee_net
#     ,sum(case when dps.is_customer_holdout is false then joker_vendor_fee_net end) non_holdout_joker_vendor_fee_net

#     ,sum(case when dps.is_customer_holdout then other_ncr_net end) holdout_other_ncr_net
#     ,sum(case when dps.is_customer_holdout is false then other_ncr_net end) non_holdout_other_ncr_net

#     ,sum(case when dps.is_customer_holdout then small_basket_fee_net end) holdout_small_basket_fee_net
#     ,sum(case when dps.is_customer_holdout is false then small_basket_fee_net end) non_holdout_small_basket_fee_net

#     ,sum(case when dps.is_customer_holdout then subscription_fee_net end) holdout_subscription_fee_net
#     ,sum(case when dps.is_customer_holdout is false then subscription_fee_net end) non_holdout_subscription_fee_net

#     ,sum(case when dps.is_customer_holdout then subscription_vendor_subsidy_net end) holdout_subscription_vendor_subsidy_net
#     ,sum(case when dps.is_customer_holdout is false then subscription_vendor_subsidy_net end) non_holdout_subscription_vendor_subsidy_net

#     ,sum(case when dps.is_customer_holdout then other_revenue_net end) holdout_other_revenue_net
#     ,sum(case when dps.is_customer_holdout is false then other_revenue_net end) non_holdout_other_revenue_net

# from fulfillment-dwh-production.curated_data_shared_mkt.bima_order_profitability op
# JOIN customer_information usi 
#     ON usi.entity_id = op.global_entity_id
# JOIN `fulfillment-dwh-production.cl.dps_sessions_mapped_to_orders` dps 
#     ON dps.entity_id = op.global_entity_id
#     and dps.platform_order_code = op.order_id
#     and dps.created_date >= '2025-01-01'
#     and dps.created_date > usi.release_date
# WHERE op.order_date > usi.release_date
#     AND op.order_date >= '2025-01-01'
#     AND dps.is_own_delivery
#     AND dps.is_sent 
#     AND vendor_vertical_parent in ('Restaurant','restaurant','restaurants')
# GROUP BY 1,2,3,4,5,6
# ),
# inter as (

#   select * 
#          ,holdout_orders / holdout_customers holdout_opu
#          ,non_holdout_orders / non_holdout_customers non_holdout_opu
#          ,flgp_holdout / holdout_orders flgpo_holdout
#          ,flgp_non_holdout / non_holdout_orders flgpo_non_holdout
#          ,(holdout_orders / holdout_customers) * non_holdout_customers adjusted_holdout_orders
#   from orders

# )
# , growth as (
#   SELECT 
#      *
#     ,non_holdout_orders - adjusted_holdout_orders incremental_orders
#     ,((non_holdout_orders - adjusted_holdout_orders) / adjusted_holdout_orders) orders_growth
#     ,(flgp_non_holdout - (flgpo_holdout * adjusted_holdout_orders)) incremental_flgp
#     ,((flgp_non_holdout - (flgpo_holdout * adjusted_holdout_orders)) / abs((flgpo_holdout * adjusted_holdout_orders))) flgp_growth
#   FROM inter o

# )
#   select *
#          ,(incremental_flgp / flgpo_non_holdout) + incremental_orders new_order_delta
#          ,((incremental_flgp / flgpo_non_holdout) + incremental_orders) / (adjusted_holdout_orders) sustainable_growth
#   from growth

# """


# sg_data = """

# WITH holdout_entities AS (
#   SELECT
#         entity_id
#        ,`Release Date` as release_date
#   FROM `logistics-data-storage-staging.long_term_pricing.global_holdout_rollout`
#   WHERE `Release Date` < DATE_SUB(CURRENT_DATE(), INTERVAL EXTRACT(DAYOFWEEK FROM CURRENT_DATE())- 2 DAY)
# ),
# orders as (
# select
#      dps.entity_id entity_id
#     ,dps.dps_customer_id customer_id
#     ,COUNT(DISTINCT case when dps.created_date <= e.release_date then dps.platform_order_code end) AS orders_dps_pre
#     ,COUNT(DISTINCT case when dps.created_date > e.release_date then dps.platform_order_code end) AS orders_dps_post
#     ,SUM(case when dps.created_date <= e.release_date then dps.fully_loaded_gross_profit_eur end) AS analytical_profit_dps_pre
#     ,SUM(case when dps.created_date > e.release_date then dps.fully_loaded_gross_profit_eur end) AS analytical_profit_dps_post
#     ,COUNT(DISTINCT case when mkt.order_date <= e.release_date then mkt.order_id end) AS orders_mkt_pre
#     ,COUNT(DISTINCT case when mkt.order_date > e.release_date then mkt.order_id end) AS orders_mkt_post
#     ,SUM(case when mkt.order_date <= e.release_date then mkt.analytical_profit end) AS analytical_profit_pre
#     ,SUM(case when mkt.order_date > e.release_date then mkt.analytical_profit end) AS analytical_profit_post
# FROM `fulfillment-dwh-production.cl.dps_sessions_mapped_to_orders` dps 
# JOIN holdout_entities AS e
#   ON dps.entity_id = e.entity_id
# LEFT JOIN fulfillment-dwh-production.curated_data_shared_mkt.bima_order_profitability mkt
#   ON mkt.global_entity_id = dps.entity_id
#   and mkt.order_id = dps.platform_order_code
#   AND order_date >= DATE_SUB(release_date, INTERVAL 8 WEEK)
#   AND order_date < CURRENT_DATE
# WHERE dps.created_date >= DATE_SUB(release_date, INTERVAL 8 WEEK)
#     AND dps.created_date < CURRENT_DATE
#     AND dps.platform_order_code IS NOT NULL
#     AND dps.is_own_delivery
#     AND dps.is_sent
#     AND vendor_vertical_parent in ('Restaurant','restaurant','restaurants')
# group by 1,2
# ),
#  customer_information AS (
#   SELECT
#          e.entity_id
#         ,e.release_date
#         ,IF(d.created_date <= e.release_date, FALSE, COALESCE(is_customer_holdout, FALSE)) AS is_customer_holdout
#         ,customer_id
#   FROM `fulfillment-dwh-production.cl.dps_holdout_users` AS d
#   JOIN holdout_entities AS e
#         ON d.entity_id = e.entity_id
#   WHERE d.created_date >= date('2025-01-01')
#     AND created_date < CURRENT_DATE()  -- data is not full/not present for current date
#     AND customer_id IS NOT NULL
#     AND customer_id NOT IN UNNEST(ARRAY(SELECT id FROM `fulfillment-dwh-production.cl._bad_dps_logs_ids`)) -- get rid of ids like 'null', 'test' etc
#   GROUP BY 1, 2, 3, 4
# ) 
#   SELECT 
#      e.entity_id
#     ,e.customer_id customer_id
#     ,e.is_customer_holdout
#     ,coalesce(orders_mkt_pre, orders_dps_pre) orders_pre
#     ,coalesce(orders_mkt_post, orders_dps_post) orders_post
#     ,coalesce(analytical_profit_pre, analytical_profit_dps_pre) analytical_profit_pre
#     ,coalesce(analytical_profit_post, analytical_profit_dps_post) analytical_profit_post
#   FROM customer_information e
#   LEFT JOIN orders o
#     ON o.customer_id = e.customer_id
#     AND o.entity_id = e.entity_id
#   WHERE e.entity_id is not null

# """



# sg_data = """

# WITH holdout_entities AS (
#   SELECT
#         entity_id
#        ,`Release Date` as release_date
#   FROM `logistics-data-storage-staging.long_term_pricing.global_holdout_rollout`
#   WHERE `Release Date` < DATE_SUB(CURRENT_DATE(), INTERVAL EXTRACT(DAYOFWEEK FROM CURRENT_DATE())- 2 DAY)
# ),
# orders as (
# select
#      dps.entity_id entity_id
#     ,dps.dps_customer_id customer_id
#     ,COUNT(DISTINCT case when dps.created_date <= e.release_date then dps.platform_order_code end) AS orders_dps_pre
#     ,COUNT(DISTINCT case when dps.created_date > e.release_date then dps.platform_order_code end) AS orders_dps_post
#     ,SUM(case when dps.created_date <= e.release_date then dps.fully_loaded_gross_profit_eur end) AS analytical_profit_dps_pre
#     ,SUM(case when dps.created_date > e.release_date then dps.fully_loaded_gross_profit_eur end) AS analytical_profit_dps_post
#     ,COUNT(DISTINCT case when mkt.order_date <= e.release_date then mkt.order_id else null end) AS orders_mkt_pre
#     ,COUNT(DISTINCT case when mkt.order_date > e.release_date then mkt.order_id else null end) AS orders_mkt_post
#     ,SUM(case when mkt.order_date <= e.release_date then mkt.analytical_profit else null end) AS analytical_profit_pre
#     ,SUM(case when mkt.order_date > e.release_date then mkt.analytical_profit else null end) AS analytical_profit_post
# FROM `fulfillment-dwh-production.cl.dps_sessions_mapped_to_orders` dps 
# JOIN holdout_entities AS e
#   ON dps.entity_id = e.entity_id
# LEFT JOIN fulfillment-dwh-production.curated_data_shared_mkt.bima_order_profitability mkt
#   ON mkt.global_entity_id = dps.entity_id
#   and mkt.order_id = dps.platform_order_code
# WHERE dps.created_date >= DATE_SUB(release_date, INTERVAL 8 WEEK)
#     AND dps.created_date < CURRENT_DATE
#     AND dps.platform_order_code IS NOT NULL
#     AND dps.is_own_delivery
#     AND dps.is_sent
#     AND vendor_vertical_parent in ('Restaurant','restaurant','restaurants')
# group by 1,2
# ),
#  customer_information AS (
#   SELECT
#          e.entity_id
#         ,e.release_date
#         ,IF(d.created_date <= e.release_date, FALSE, COALESCE(is_customer_holdout, FALSE)) AS is_customer_holdout
#         ,customer_id
#   FROM `fulfillment-dwh-production.cl.dps_holdout_users` AS d
#   JOIN holdout_entities AS e
#         ON d.entity_id = e.entity_id
#   WHERE d.created_date >= date('2025-01-01')
#     AND created_date < CURRENT_DATE()  -- data is not full/not present for current date
#     AND customer_id IS NOT NULL
#     AND customer_id NOT IN UNNEST(ARRAY(SELECT id FROM `fulfillment-dwh-production.cl._bad_dps_logs_ids`)) -- get rid of ids like 'null', 'test' etc
#   GROUP BY 1, 2, 3, 4
# ) 
#   SELECT 
#      e.entity_id
#     ,e.customer_id customer_id
#     ,e.is_customer_holdout
#     ,orders_mkt_pre 
#     ,orders_dps_pre
#     ,orders_mkt_post
#     ,orders_dps_post
#     ,analytical_profit_pre
#     ,analytical_profit_dps_pre
#     ,analytical_profit_post
#     ,analytical_profit_dps_post
#   FROM customer_information e
#   LEFT JOIN orders o
#     ON o.customer_id = e.customer_id
#     AND o.entity_id = e.entity_id
#   WHERE e.entity_id is not null


# # """


# sg_data = """

# WITH holdout_entities AS (
#   SELECT
#         entity_id
#        ,`Release Date` as release_date
#   FROM `logistics-data-storage-staging.long_term_pricing.global_holdout_rollout`
#   WHERE `Release Date` < DATE_SUB(CURRENT_DATE(), INTERVAL EXTRACT(DAYOFWEEK FROM CURRENT_DATE())- 2 DAY)
# ),
# orders as (
# select
#      dps.entity_id entity_id
#     ,dps.dps_customer_id customer_id
#     ,COUNT(DISTINCT case when dps.created_date <= e.release_date then dps.platform_order_code end) AS orders_dps_pre
#     ,COUNT(DISTINCT case when dps.created_date > e.release_date then dps.platform_order_code end) AS orders_dps_post
#     ,SUM(case when dps.created_date <= e.release_date then dps.fully_loaded_gross_profit_eur end) AS analytical_profit_dps_pre
#     ,SUM(case when dps.created_date > e.release_date then dps.fully_loaded_gross_profit_eur end) AS analytical_profit_dps_post
#     ,COUNT(DISTINCT case when mkt.order_date <= e.release_date then mkt.order_id end) AS orders_mkt_pre
#     ,COUNT(DISTINCT case when mkt.order_date > e.release_date then mkt.order_id end) AS orders_mkt_post
#     ,SUM(case when mkt.order_date <= e.release_date then mkt.analytical_profit end) AS analytical_profit_pre
#     ,SUM(case when mkt.order_date > e.release_date then mkt.analytical_profit end) AS analytical_profit_post
# FROM `fulfillment-dwh-production.cl.dps_sessions_mapped_to_orders` dps 
# JOIN holdout_entities AS e
#   ON dps.entity_id = e.entity_id
# LEFT JOIN fulfillment-dwh-production.curated_data_shared_mkt.bima_order_profitability mkt
#   ON mkt.global_entity_id = dps.entity_id
#   and mkt.order_id = dps.platform_order_code
# WHERE dps.created_date >= DATE_SUB(release_date, INTERVAL 8 WEEK)
#     AND dps.created_date < CURRENT_DATE
#     AND dps.platform_order_code IS NOT NULL
#     AND dps.is_own_delivery
#     AND dps.is_sent
#     AND vendor_vertical_parent in ('Restaurant','restaurant','restaurants')
#     AND dps.entity_id not in ('FP_TH' ,'FP_TW','YS_TR','AP_PA','FP_PH','FP_MY','PY_AR','FP_SG','PY_CL','OP_SE','PO_FI','EF_GR','PY_EC','PY_UY','NP_HU','MJM_AT','FP_LA','DJ_CZ','PY_DO','PY_PE','FP_BD','FP_MM','PY_PY','PY_SV','PY_NI','FY_CY','PY_HN','PY_BO','PY_GT','FO_NO','PY_CR','PY_VE')
#     AND mkt.global_entity_id not in ('FP_TH' ,'FP_TW','YS_TR','AP_PA','FP_PH','FP_MY','PY_AR','FP_SG','PY_CL','OP_SE','PO_FI','EF_GR','PY_EC','PY_UY','NP_HU','MJM_AT','FP_LA','DJ_CZ','PY_DO','PY_PE','FP_BD','FP_MM','PY_PY','PY_SV','PY_NI','FY_CY','PY_HN','PY_BO','PY_GT','FO_NO','PY_CR','PY_VE')
# group by 1,2
# ),
#  customer_information AS (
#   SELECT
#          e.entity_id
#         ,e.release_date
#         ,IF(d.created_date <= e.release_date, FALSE, COALESCE(is_customer_holdout, FALSE)) AS is_customer_holdout
#         ,customer_id
#   FROM `fulfillment-dwh-production.cl.dps_holdout_users` AS d
#   JOIN holdout_entities AS e
#         ON d.entity_id = e.entity_id
#   WHERE d.created_date >= date('2025-01-01')
#     AND created_date < CURRENT_DATE()  -- data is not full/not present for current date
#     AND customer_id IS NOT NULL
#     AND customer_id NOT IN UNNEST(ARRAY(SELECT id FROM `fulfillment-dwh-production.cl._bad_dps_logs_ids`)) -- get rid of ids like 'null', 'test' etc
#     AND d.entity_id not in ('FP_TH' ,'FP_TW','YS_TR','AP_PA','FP_PH','FP_MY','PY_AR','FP_SG','PY_CL','OP_SE','PO_FI','EF_GR','PY_EC','PY_UY','NP_HU','MJM_AT','FP_LA','DJ_CZ','PY_DO','PY_PE','FP_BD','FP_MM','PY_PY','PY_SV','PY_NI','FY_CY','PY_HN','PY_BO','PY_GT','FO_NO','PY_CR','PY_VE')
#   GROUP BY 1, 2, 3, 4
# ) 
#   SELECT 
#      e.entity_id
#     ,e.customer_id customer_id
#     ,e.is_customer_holdout
#     ,orders_mkt_pre 
#     ,orders_mkt_post
#     ,orders_dps_pre
#     ,orders_dps_post
#     ,analytical_profit_pre
#     ,analytical_profit_post
#     ,analytical_profit_dps_pre
#     ,analytical_profit_dps_post
#   FROM customer_information e
#   LEFT JOIN orders o
#     ON o.customer_id = e.customer_id
#     AND o.entity_id = e.entity_id
#   WHERE e.entity_id is not null


# """


sg_data = """

WITH holdout_entities AS (
  SELECT
        entity_id
       ,`Release Date` as release_date
  FROM `logistics-data-storage-staging.long_term_pricing.global_holdout_rollout`
  WHERE `Release Date` < DATE_SUB(CURRENT_DATE(), INTERVAL EXTRACT(DAYOFWEEK FROM CURRENT_DATE())- 2 DAY)
  AND entity_id not in ('FP_TH' ,'FP_TW','YS_TR','AP_PA','FP_PH','FP_MY','PY_AR','FP_SG','PY_CL','OP_SE','PO_FI','EF_GR','PY_EC','PY_UY','NP_HU','MJM_AT','FP_LA','DJ_CZ','PY_DO','PY_PE','FP_BD','FP_MM','PY_PY','PY_SV','PY_NI','FY_CY','PY_HN','PY_BO','PY_GT','FO_NO','PY_CR','PY_VE')
),
orders as (
select
     dps.entity_id entity_id
    ,dps.dps_customer_id customer_id
    ,COUNT(DISTINCT case when dps.created_date <= e.release_date then dps.platform_order_code end) AS orders_dps_pre
    ,COUNT(DISTINCT case when dps.created_date > e.release_date then dps.platform_order_code end) AS orders_dps_post
    ,SUM(case when dps.created_date <= e.release_date then dps.fully_loaded_gross_profit_eur end) AS analytical_profit_dps_pre
    ,SUM(case when dps.created_date > e.release_date then dps.fully_loaded_gross_profit_eur end) AS analytical_profit_dps_post
FROM `fulfillment-dwh-production.cl.dps_sessions_mapped_to_orders` dps 
JOIN holdout_entities AS e
  ON dps.entity_id = e.entity_id
WHERE dps.created_date >= DATE_SUB(release_date, INTERVAL 8 WEEK)
    AND dps.created_date < CURRENT_DATE
    AND dps.platform_order_code IS NOT NULL
    AND dps.is_own_delivery
    AND dps.is_sent
    AND dps.entity_id not in ('FP_TH' ,'FP_TW','YS_TR','AP_PA','FP_PH','FP_MY','PY_AR','FP_SG','PY_CL','OP_SE','PO_FI','EF_GR','PY_EC','PY_UY','NP_HU','MJM_AT','FP_LA','DJ_CZ','PY_DO','PY_PE','FP_BD','FP_MM','PY_PY','PY_SV','PY_NI','FY_CY','PY_HN','PY_BO','PY_GT','FO_NO','PY_CR','PY_VE')
    AND vendor_vertical_parent in ('Restaurant','restaurant','restaurants')
group by 1,2
),
 customer_information AS (
  SELECT
         e.entity_id
        ,e.release_date
        ,IF(d.created_date <= e.release_date, FALSE, COALESCE(is_customer_holdout, FALSE)) AS is_customer_holdout
        ,customer_id
  FROM `fulfillment-dwh-production.cl.dps_holdout_users` AS d
  JOIN holdout_entities AS e
        ON d.entity_id = e.entity_id
  WHERE d.created_date >= date('2025-01-01')
    AND created_date < CURRENT_DATE()  -- data is not full/not present for current date
    AND customer_id IS NOT NULL
    AND customer_id NOT IN UNNEST(ARRAY(SELECT id FROM `fulfillment-dwh-production.cl._bad_dps_logs_ids`)) -- get rid of ids like 'null', 'test' etc
    AND e.entity_id not in ('FP_TH' ,'FP_TW','YS_TR','AP_PA','FP_PH','FP_MY','PY_AR','FP_SG','PY_CL','OP_SE','PO_FI','EF_GR','PY_EC','PY_UY','NP_HU','MJM_AT','FP_LA','DJ_CZ','PY_DO','PY_PE','FP_BD','FP_MM','PY_PY','PY_SV','PY_NI','FY_CY','PY_HN','PY_BO','PY_GT','FO_NO','PY_CR','PY_VE')
  GROUP BY 1, 2, 3, 4
) 
  SELECT 
     e.entity_id
    ,e.customer_id customer_id
    ,e.is_customer_holdout
    ,orders_dps_pre orders_pre
    ,orders_dps_post orders_post
    ,analytical_profit_dps_pre analytical_profit_pre
    ,analytical_profit_dps_post analytical_profit_post
  FROM customer_information e
  LEFT JOIN orders o
    ON o.customer_id = e.customer_id
    AND o.entity_id = e.entity_id
  WHERE e.entity_id is not null



"""

In [106]:
project_id = "logistics-customer-staging"
logging.info(f"Initializing BigQuery client for project: {project_id}")

try:
    # Initialize BigQuery client
    client = bigquery.Client(project=project_id)
except Exception as e:
    logging.error(f"Failed to initialize BigQuery client: {e}")
    sys.exit(1)

#growth_df = client.query(growth_data).to_dataframe()
#mkt_df =  client.query(mkt_data).to_dataframe()

sg_df = client.query(sg_data).to_dataframe()

2025-02-14 17:01:02,474 INFO:Initializing BigQuery client for project: logistics-customer-staging


/Users/shazeb.asad/global_pricing/.venv/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


2025-02-14 17:01:03,105 WARNING:No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable


In [107]:
# sg_df['orders_pre'] = sg_df['orders_mkt_pre']
# sg_df.loc[(sg_df['orders_pre'].isna()) | (sg_df['orders_pre'] == 0), 'orders_pre'] = sg_df['orders_dps_pre']

# sg_df['orders_post'] = sg_df['orders_mkt_post']
# sg_df.loc[(sg_df['orders_post'].isna()) | (sg_df['orders_post'] == 0), 'orders_post'] = sg_df['orders_dps_post']

# sg_df['analytical_profit_pre'] = sg_df['analytical_profit_pre']
# sg_df.loc[(sg_df['analytical_profit_pre'].isna()) | (sg_df['analytical_profit_pre'] == 0), 'analytical_profit_pre'] = sg_df['analytical_profit_dps_pre']

# sg_df['analytical_profit_post'] = sg_df['analytical_profit_post']
# sg_df.loc[(sg_df['analytical_profit_post'].isna()) | (sg_df['analytical_profit_post'] == 0), 'analytical_profit_post'] = sg_df['analytical_profit_dps_post']



In [108]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind

def calculate_sustainable_growth(data, pre_profit, post_profit):
    
    data = data.copy()

    entity = data['entity_id'].iloc[0]

    # Apply CUPED to FLGP
    data_flgp = data.dropna(subset=[pre_profit, post_profit])
    theta_flgp = np.cov(data_flgp[pre_profit], data_flgp[post_profit])[0, 1] / np.var(data_flgp[pre_profit])
    data_flgp['flgp_post_cuped'] = data_flgp[post_profit] - theta_flgp * (data_flgp[pre_profit] - data_flgp[pre_profit].mean())

    # Apply CUPED to Orders
    data_orders = data.dropna(subset=['orders_pre', 'orders_post'])
    theta_orders = np.cov(data_orders['orders_pre'], data_orders['orders_post'])[0, 1] / np.var(data_orders['orders_pre'])
    data_orders['orders_post_cuped'] = data_orders['orders_post'] - theta_orders * (data_orders['orders_pre'] - data_orders['orders_pre'].mean())

    # Total FLGP and Orders (CUPED)
    holdout_total_flgp_cuped = data_flgp.loc[data_flgp['is_customer_holdout'], 'flgp_post_cuped'].sum()
    non_holdout_total_flgp_cuped = data_flgp.loc[~data_flgp['is_customer_holdout'], 'flgp_post_cuped'].sum()

    holdout_total_orders_cuped = data_orders.loc[data_orders['is_customer_holdout'], 'orders_post_cuped'].sum()
    non_holdout_total_orders_cuped = data_orders.loc[~data_orders['is_customer_holdout'], 'orders_post_cuped'].sum()

    # Normalize for Population Differences
    holdout_user_count = data['is_customer_holdout'].sum()
    non_holdout_user_count = (~data['is_customer_holdout']).sum()

    scaled_holdout_total_flgp_cuped = (holdout_total_flgp_cuped / holdout_user_count) * non_holdout_user_count if holdout_user_count != 0 else np.nan
    scaled_holdout_total_orders_cuped = (holdout_total_orders_cuped / holdout_user_count) * non_holdout_user_count if holdout_user_count != 0 else np.nan

    # FLGP per Order and Orders per User (CUPED)
    holdout_flgp_per_order_cuped = (
        np.nan if np.isnan(holdout_total_orders_cuped) or holdout_total_orders_cuped == 0
        else holdout_total_flgp_cuped / holdout_total_orders_cuped
    )
    non_holdout_flgp_per_order_cuped = (
    np.nan if np.isnan(non_holdout_total_orders_cuped) or non_holdout_total_orders_cuped == 0
    else non_holdout_total_flgp_cuped / non_holdout_total_orders_cuped
    )

    holdout_orders_per_user_cuped = holdout_total_orders_cuped / holdout_user_count if holdout_user_count != 0 else np.nan
    non_holdout_orders_per_user_cuped = non_holdout_total_orders_cuped / non_holdout_user_count if non_holdout_user_count != 0 else np.nan

    # Incremental Differences (CUPED)
    incremental_orders_cuped = non_holdout_total_orders_cuped - scaled_holdout_total_orders_cuped
    incremental_flgp_cuped = non_holdout_total_flgp_cuped - scaled_holdout_total_flgp_cuped

    # Percentage Changes (CUPED)
    percentage_change_orders_cuped = ((incremental_orders_cuped) / abs(scaled_holdout_total_orders_cuped)) * 100 if scaled_holdout_total_orders_cuped != 0 else np.nan
    percentage_change_flgp_cuped = ((incremental_flgp_cuped) / abs(scaled_holdout_total_flgp_cuped)) * 100 if scaled_holdout_total_flgp_cuped != 0 else np.nan

    # Sustainable Growth Calculation
    sustainable_growth = ((incremental_orders_cuped + (incremental_flgp_cuped / non_holdout_flgp_per_order_cuped)) / scaled_holdout_total_orders_cuped) * 100 if scaled_holdout_total_orders_cuped != 0 else np.nan

    # T-tests for significance
    t_stat_orders, p_value_orders = ttest_ind(
        data_orders.loc[data_orders['is_customer_holdout'], 'orders_post_cuped'],
        data_orders.loc[~data_orders['is_customer_holdout'], 'orders_post_cuped'],
        equal_var=False
    )

    t_stat_flgp, p_value_flgp = ttest_ind(
        data_flgp.loc[data_flgp['is_customer_holdout'], 'flgp_post_cuped'],
        data_flgp.loc[~data_flgp['is_customer_holdout'], 'flgp_post_cuped'],
        equal_var=False
    )

    return {
        'entity': entity,
        'sustainable_growth': sustainable_growth,
        'percentage_change_orders_cuped': percentage_change_orders_cuped,
        'incremental_orders_cuped': incremental_orders_cuped,
        'non_holdout_total_orders_cuped':non_holdout_total_orders_cuped,
        'holdout_total_orders_cuped':scaled_holdout_total_orders_cuped,
        't_stat_orders': t_stat_orders,
        'p_value_orders': p_value_orders,
        'percentage_change_flgp_cuped': percentage_change_flgp_cuped,
        'incremental_flgp_cuped': incremental_flgp_cuped,
        'non_holdout_total_flgp_cuped': non_holdout_total_flgp_cuped,
        'holdout_total_flgp_cuped':scaled_holdout_total_flgp_cuped,
        't_stat_flgp': t_stat_flgp,
        'p_value_flgp': p_value_flgp,
        'holdout_flgp_per_order_cuped': holdout_flgp_per_order_cuped,
        'non_holdout_flgp_per_order_cuped': non_holdout_flgp_per_order_cuped,
        'holdout_orders_per_user_cuped': holdout_orders_per_user_cuped,
        'non_holdout_orders_per_user_cuped': non_holdout_orders_per_user_cuped
    }


# results = []
# for entity in mkt_df['entity_id'].unique():
#     entity_data = mkt_df[mkt_df['entity_id'] == entity]
#     result = calculate_sustainable_growth(entity_data, 'local_analytical_profit_pre','local_analytical_profit_post')
#     result['entity_id'] = entity
#     results.append(result)

# pd.DataFrame(results)

# mkt_df['profit_pre'] = mkt_df['revenue_pre'] + mkt_df['cost_of_sales_pre']
# mkt_df['profit_post'] = mkt_df['revenue_pre'] + mkt_df['cost_of_sales_pre']

# Define the list of pre and post metrics
metric_pairs = [
    ('analytical_profit_pre', 'analytical_profit_post'),
]

# results = []

# # Loop through each metric pair
# for pre_metric, post_metric in metric_pairs:
#     for entity in sg_df['entity_id'].unique():
#         entity_data = sg_df[sg_df['entity_id'] == entity]
#         result = calculate_sustainable_growth(entity_data, pre_metric, post_metric)
        
#         # Add entity_id and metric information to the result
#         result['metric_used'] = f"{pre_metric}_vs_{post_metric}"
        
#         results.append(result)

# # Convert results to DataFrame
# final_results_df = pd.DataFrame(results)


# final_results_df = final_results_df.dropna(subset= 'sustainable_growth')

# final_results_df.to_csv('profitable_growth.csv', index=False) 

# Initialize results list
results = []

# Loop through each metric pair
for pre_metric, post_metric in metric_pairs:
    for entity in sg_df['entity_id'].unique():
        entity_data = sg_df[sg_df['entity_id'] == entity]
        
        try:
            # Call function and process results
            result = calculate_sustainable_growth(entity_data, pre_metric, post_metric)
            result['metric_used'] = f"{pre_metric}_vs_{post_metric}"
            results.append(result)
        
        except Exception as e:
            print(f"Skipping entity {entity} due to error: {e}")
            continue  # Skip to the next entity

# Convert results to DataFrame
final_results_df = pd.DataFrame(results)

# Drop rows where sustainable_growth is NaN
final_results_df = final_results_df.dropna(subset=['sustainable_growth'])

# Save to CSV
final_results_df.to_csv('profitable_growth.csv', index=False)

print("Processing completed. Check profitable_growth.csv for results.")

/var/folders/7c/hjrbzbpn61jgnsn14y5f3l7c0000gq/T/ipykernel_69416/2572999320.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_flgp['flgp_post_cuped'] = data_flgp[post_profit] - theta_flgp * (data_flgp[pre_profit] - data_flgp[pre_profit].mean())
/var/folders/7c/hjrbzbpn61jgnsn14y5f3l7c0000gq/T/ipykernel_69416/2572999320.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_orders['orders_post_cuped'] = data_orders['orders_post'] - theta_orders * (data_orders['orders_pre'] - data_orders['orders_pr

Processing completed. Check profitable_growth.csv for results.


/Users/shazeb.asad/global_pricing/.venv/lib/python3.12/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Users/shazeb.asad/global_pricing/.venv/lib/python3.12/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/var/folders/7c/hjrbzbpn61jgnsn14y5f3l7c0000gq/T/ipykernel_69416/2572999320.py:13: RuntimeWarning: Degrees of freedom <= 0 for slice
  theta_flgp = np.cov(data_flgp[pre_profit], data_flgp[post_profit])[0, 1] / np.var(data_flgp[pre_profit])
/Users/shazeb.asad/global_pricing/.venv/lib/python3.12/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/Users/shazeb.asad/global_pricing/.venv/lib/python3.12/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/Users/shazeb.asad/global_pricing/.venv/

In [109]:
final_results_df

,entity,sustainable_growth,percentage_change_orders_cuped,incremental_orders_cuped,non_holdout_total_orders_cuped,holdout_total_orders_cuped,t_stat_orders,p_value_orders,percentage_change_flgp_cuped,incremental_flgp_cuped,non_holdout_total_flgp_cuped,holdout_total_flgp_cuped,t_stat_flgp,p_value_flgp,holdout_flgp_per_order_cuped,non_holdout_flgp_per_order_cuped,holdout_orders_per_user_cuped,non_holdout_orders_per_user_cuped,metric_used
0,HF_EG,12.652320,5.624633,418689.402262,7.862542e+06,7.443852e+06,-8.984931,2.694806e-19,7.127693,224986.821859,3.381504e+06,3.156517e+06,-5.461298,4.773979e-08,0.424043,0.430078,2.239111,2.365053,analytical_profit_pre_vs_analytical_profit_post
1,TB_IQ,8.071194,3.184733,67440.284206,2.185052e+06,2.117612e+06,-2.983866,2.851123e-03,4.971055,67032.655379,1.415492e+06,1.348459e+06,-1.986907,4.696586e-02,0.636783,0.647807,2.053400,2.118796,analytical_profit_pre_vs_analytical_profit_post
2,TB_QA,-0.721610,0.077008,2049.045062,2.662884e+06,2.660835e+06,-0.136515,8.914162e-01,-0.791685,-27056.250231,3.390496e+06,3.417553e+06,0.176862,8.596215e-01,1.284391,1.273242,3.341502,3.344075,analytical_profit_pre_vs_analytical_profit_post
3,TB_AE,2.792687,1.458131,147703.610305,1.027735e+07,1.012965e+07,-3.080046,2.071090e-03,1.332909,323189.362095,2.457012e+07,2.424693e+07,-3.619941,2.952399e-04,2.393659,2.390704,3.762943,3.817812,analytical_profit_pre_vs_analytical_profit_post
4,TB_OM,2.403548,-0.422707,-8473.054819,1.995999e+06,2.004473e+06,-0.469791,6.385146e-01,2.921162,36465.595927,1.284790e+06,1.248325e+06,-1.001636,3.165566e-01,0.622770,0.643683,3.004112,2.991413,analytical_profit_pre_vs_analytical_profit_post
5,GV_MD,-6.396611,-3.669099,-4694.756001,1.232592e+05,1.279539e+05,0.786639,4.316314e-01,-2.753437,-3747.005095,1.323376e+05,1.360846e+05,-0.299084,7.649778e-01,1.063544,1.073653,1.161760,1.119134,analytical_profit_pre_vs_analytical_profit_post
6,GV_RO,5.806963,2.503453,48173.030867,1.972437e+06,1.924264e+06,-1.699684,8.920700e-02,3.330154,93349.504008,2.896509e+06,2.803159e+06,-1.647752,9.944038e-02,1.456744,1.468493,1.287816,1.320056,analytical_profit_pre_vs_analytical_profit_post
7,TB_KW,5.650179,1.639130,83823.937911,5.197752e+06,5.113928e+06,-2.651003,8.031673e-03,4.108500,323519.313500,8.197910e+06,7.874391e+06,-3.123895,1.788322e-03,1.539793,1.577203,4.176800,4.245264,analytical_profit_pre_vs_analytical_profit_post
8,HS_SA,0.580731,2.116875,117633.338400,5.674567e+06,5.556933e+06,-4.750941,2.028695e-06,-1.482006,-91094.617964,6.055617e+06,6.146712e+06,1.232326,2.178368e-01,1.106134,1.067151,0.378901,0.386922,analytical_profit_pre_vs_analytical_profit_post
9,TB_BH,3.821399,2.948410,60196.109176,2.101843e+06,2.041646e+06,-2.017075,4.372538e-02,0.855239,23934.608907,2.822523e+06,2.798588e+06,0.266844,7.896012e-01,1.370751,1.342880,4.333804,4.461582,analytical_profit_pre_vs_analytical_profit_post


In [61]:
test  =  sg_df[sg_df['entity_id'] == 'PO_FI']

In [63]:
test

,entity_id,customer_id,is_customer_holdout,orders_pre,orders_post,analytical_profit_pre,analytical_profit_post
25,PO_FI,po1obh1z,False,24,4,-17.949185,9.762005
156,PO_FI,pohqw6hy,False,23,3,-42.375254,1.006827
239,PO_FI,poe3jx0s,False,28,15,-42.902671,43.197208
2698,PO_FI,pol95wg0,True,18,22,-41.831534,-62.586402
4286,PO_FI,poclg1qy,False,23,11,-12.544819,2.854819
...,...,...,...,...,...,...,...
93886682,PO_FI,pokch5yc,False,11,7,-67.490966,-34.065480
93887147,PO_FI,popkz7db,False,11,13,-3.152952,5.427068
93887228,PO_FI,poelv3hc,False,11,2,1.840586,2.155293
93887273,PO_FI,pouhq8ip,False,11,8,-48.543584,-9.720950


In [110]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind

def calculate_sustainable_growth(data, pre_profit, post_profit):
    
    data = data.copy()

    entity = data['entity_id'].iloc[0]

    # Apply CUPED to FLGP
    data_flgp = data.dropna(subset=[pre_profit, post_profit])
    theta_flgp = np.cov(data_flgp[pre_profit], data_flgp[post_profit])[0, 1] / np.var(data_flgp[pre_profit])
    data_flgp['flgp_post_cuped'] = data_flgp[post_profit] - theta_flgp * (data_flgp[pre_profit] - data_flgp[pre_profit].mean())

    # Apply CUPED to Orders
    data_orders = data.dropna(subset=['orders_pre', 'orders_post'])
    theta_orders = np.cov(data_orders['orders_pre'], data_orders['orders_post'])[0, 1] / np.var(data_orders['orders_pre'])
    data_orders['orders_post_cuped'] = data_orders['orders_post'] - theta_orders * (data_orders['orders_pre'] - data_orders['orders_pre'].mean())

    # Per User Metrics
    holdout_flgpu_post = data_flgp.loc[data_flgp['is_customer_holdout'], 'flgp_post_cuped'].mean()
    holdout_flgpu_pre = data_flgp.loc[data_flgp['is_customer_holdout'], pre_profit].mean()
    
    non_holdout_flgpu_post = data_flgp.loc[~data_flgp['is_customer_holdout'], 'flgp_post_cuped'].mean()
    non_holdout_flgpu_pre = data_flgp.loc[~data_flgp['is_customer_holdout'], pre_profit].mean()

    holdout_orders_per_user_post = data_orders.loc[data_orders['is_customer_holdout'], 'orders_post_cuped'].mean()
    holdout_orders_per_user_pre = data_orders.loc[data_orders['is_customer_holdout'], 'orders_pre'].mean()
    
    non_holdout_orders_per_user_post = data_orders.loc[~data_orders['is_customer_holdout'], 'orders_post_cuped'].mean()
    non_holdout_orders_per_user_pre = data_orders.loc[~data_orders['is_customer_holdout'], 'orders_pre'].mean()

    # Apply DID
    # DID to flgpu
    d_flgpu_pre = non_holdout_flgpu_pre - holdout_flgpu_pre
    d_flgpu_post = non_holdout_flgpu_post - holdout_flgpu_post
    did_flgpu  = d_flgpu_post - d_flgpu_pre

    non_holdout_flgpu_adjusted_post = non_holdout_flgpu_post - did_flgpu
    holdout_flgpu_adjusted_post = holdout_flgpu_post

    # DID to orders_per_user
    d_orders_per_user_pre = non_holdout_orders_per_user_pre - holdout_orders_per_user_pre
    d_orders_per_user_post = non_holdout_orders_per_user_post - holdout_orders_per_user_post
    did_orders_per_user  = d_orders_per_user_post - d_orders_per_user_pre

    non_holdout_orders_per_user_adjusted_post = non_holdout_orders_per_user_post - did_orders_per_user
    holdout_orders_per_user_adjusted_post = holdout_orders_per_user_post

    # Total FLGP and Orders
    holdout_user_count = data['is_customer_holdout'].sum()
    non_holdout_user_count = (~data['is_customer_holdout']).sum()

    holdout_total_flgp_cuped = holdout_flgpu_adjusted_post * holdout_user_count
    non_holdout_total_flgp_cuped = non_holdout_flgpu_adjusted_post * non_holdout_user_count

    holdout_total_orders_cuped = holdout_orders_per_user_adjusted_post * holdout_user_count
    non_holdout_total_orders_cuped = non_holdout_orders_per_user_adjusted_post * non_holdout_user_count

    # Normalize for Population Differences
    holdout_user_count = data['is_customer_holdout'].sum()
    non_holdout_user_count = (~data['is_customer_holdout']).sum()

    scaled_holdout_total_flgp_cuped = (holdout_total_flgp_cuped / holdout_user_count) * non_holdout_user_count if holdout_user_count != 0 else np.nan
    scaled_holdout_total_orders_cuped = (holdout_total_orders_cuped / holdout_user_count) * non_holdout_user_count if holdout_user_count != 0 else np.nan

    holdout_flgp_per_order_cuped = (
    np.nan if pd.isna(holdout_total_orders_cuped) or holdout_total_orders_cuped == 0
    else holdout_total_flgp_cuped / holdout_total_orders_cuped
    )
    non_holdout_flgp_per_order_cuped = (
    np.nan if pd.isna(non_holdout_total_orders_cuped) or non_holdout_total_orders_cuped == 0
    else non_holdout_total_flgp_cuped / non_holdout_total_orders_cuped
    )

    holdout_orders_per_user_cuped = holdout_orders_per_user_adjusted_post
    non_holdout_orders_per_user_cuped = non_holdout_orders_per_user_adjusted_post

    # Incremental Differences (CUPED)
    incremental_orders_cuped = non_holdout_total_orders_cuped - scaled_holdout_total_orders_cuped
    incremental_flgp_cuped = non_holdout_total_flgp_cuped - scaled_holdout_total_flgp_cuped

    # Percentage Changes (CUPED)
    percentage_change_orders_cuped = ((incremental_orders_cuped) / abs(scaled_holdout_total_orders_cuped)) * 100 if scaled_holdout_total_orders_cuped != 0 else np.nan
    percentage_change_flgp_cuped = ((incremental_flgp_cuped) / abs(scaled_holdout_total_flgp_cuped)) * 100 if scaled_holdout_total_flgp_cuped != 0 else np.nan

    # Sustainable Growth Calculation
    sustainable_growth = ((incremental_orders_cuped + (incremental_flgp_cuped / non_holdout_flgp_per_order_cuped)) / scaled_holdout_total_orders_cuped) * 100 if scaled_holdout_total_orders_cuped != 0 else np.nan

    # T-tests for significance
    t_stat_orders, p_value_orders = ttest_ind(
        data_orders.loc[data_orders['is_customer_holdout'], 'orders_post_cuped'],
        data_orders.loc[~data_orders['is_customer_holdout'], 'orders_post_cuped'],
        equal_var=False
    )

    t_stat_flgp, p_value_flgp = ttest_ind(
        data_flgp.loc[data_flgp['is_customer_holdout'], 'flgp_post_cuped'],
        data_flgp.loc[~data_flgp['is_customer_holdout'], 'flgp_post_cuped'],
        equal_var=False
    )

    return {
        'entity': entity,
        'sustainable_growth': sustainable_growth,
        'percentage_change_orders_cuped': percentage_change_orders_cuped,
        'incremental_orders_cuped': incremental_orders_cuped,
        'non_holdout_total_orders_cuped':non_holdout_total_orders_cuped,
        'holdout_total_orders_cuped':scaled_holdout_total_orders_cuped,
        't_stat_orders': t_stat_orders,
        'p_value_orders': p_value_orders,
        'percentage_change_flgp_cuped': percentage_change_flgp_cuped,
        'incremental_flgp_cuped': incremental_flgp_cuped,
        'non_holdout_total_flgp_cuped': non_holdout_total_flgp_cuped,
        'holdout_total_flgp_cuped':scaled_holdout_total_flgp_cuped,
        't_stat_flgp': t_stat_flgp,
        'p_value_flgp': p_value_flgp,
        'holdout_flgp_per_order_cuped': holdout_flgp_per_order_cuped,
        'non_holdout_flgp_per_order_cuped': non_holdout_flgp_per_order_cuped,
        'holdout_orders_per_user_cuped': holdout_orders_per_user_cuped,
        'non_holdout_orders_per_user_cuped': non_holdout_orders_per_user_cuped
    }


# results = []
# for entity in mkt_df['entity_id'].unique():
#     entity_data = mkt_df[mkt_df['entity_id'] == entity]
#     result = calculate_sustainable_growth(entity_data, 'local_analytical_profit_pre','local_analytical_profit_post')
#     result['entity_id'] = entity
#     results.append(result)

# pd.DataFrame(results)

# mkt_df['profit_pre'] = mkt_df['revenue_pre'] + mkt_df['cost_of_sales_pre']
# mkt_df['profit_post'] = mkt_df['revenue_pre'] + mkt_df['cost_of_sales_pre']

# Define the list of pre and post metrics
metric_pairs = [
    ('analytical_profit_pre', 'analytical_profit_post'),
]

# results = []

# # Loop through each metric pair
# for pre_metric, post_metric in metric_pairs:
#     for entity in sg_df['entity_id'].unique():
#         entity_data = sg_df[sg_df['entity_id'] == entity]
#         result = calculate_sustainable_growth(entity_data, pre_metric, post_metric)
#         # Add entity_id and metric information to the result
#         result['metric_used'] = f"{pre_metric}_vs_{post_metric}"
        
#         results.append(result)

# # Convert results to DataFrame
# final_results_df = pd.DataFrame(results)


# final_results_df = final_results_df.dropna(subset= 'sustainable_growth')

# final_results_df.to_csv('profitable_growth.csv', index=False) 


# Initialize results list
results = []

# Loop through each metric pair
for pre_metric, post_metric in metric_pairs:
    for entity in sg_df['entity_id'].unique():
        entity_data = sg_df[sg_df['entity_id'] == entity]
        
        try:
            # Call function and process results
            result = calculate_sustainable_growth(entity_data, pre_metric, post_metric)
            result['metric_used'] = f"{pre_metric}_vs_{post_metric}"
            results.append(result)
        
        except Exception as e:
            print(f"Skipping entity {entity} due to error: {e}")
            continue  # Skip to the next entity

# Convert results to DataFrame
final_results_df = pd.DataFrame(results)

# Drop rows where sustainable_growth is NaN
final_results_df = final_results_df.dropna(subset=['sustainable_growth'])

# Save to CSV
final_results_df.to_csv('profitable_growth.csv', index=False)

print("Processing completed. Check profitable_growth.csv for results.")


/var/folders/7c/hjrbzbpn61jgnsn14y5f3l7c0000gq/T/ipykernel_69416/2740122075.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_flgp['flgp_post_cuped'] = data_flgp[post_profit] - theta_flgp * (data_flgp[pre_profit] - data_flgp[pre_profit].mean())
/var/folders/7c/hjrbzbpn61jgnsn14y5f3l7c0000gq/T/ipykernel_69416/2740122075.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_orders['orders_post_cuped'] = data_orders['orders_post'] - theta_orders * (data_orders['orders_pre'] - data_orders['orders_pr

Skipping entity GV_MA due to error: boolean value of NA is ambiguous
Skipping entity GV_TN due to error: boolean value of NA is ambiguous
Processing completed. Check profitable_growth.csv for results.


/Users/shazeb.asad/global_pricing/.venv/lib/python3.12/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Users/shazeb.asad/global_pricing/.venv/lib/python3.12/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/var/folders/7c/hjrbzbpn61jgnsn14y5f3l7c0000gq/T/ipykernel_69416/2740122075.py:13: RuntimeWarning: Degrees of freedom <= 0 for slice
  theta_flgp = np.cov(data_flgp[pre_profit], data_flgp[post_profit])[0, 1] / np.var(data_flgp[pre_profit])
/Users/shazeb.asad/global_pricing/.venv/lib/python3.12/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/Users/shazeb.asad/global_pricing/.venv/lib/python3.12/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/Users/shazeb.asad/global_pricing/.venv/

In [111]:
final_results_df

,entity,sustainable_growth,percentage_change_orders_cuped,incremental_orders_cuped,non_holdout_total_orders_cuped,holdout_total_orders_cuped,t_stat_orders,p_value_orders,percentage_change_flgp_cuped,incremental_flgp_cuped,non_holdout_total_flgp_cuped,holdout_total_flgp_cuped,t_stat_flgp,p_value_flgp,holdout_flgp_per_order_cuped,non_holdout_flgp_per_order_cuped,holdout_orders_per_user_cuped,non_holdout_orders_per_user_cuped,metric_used
0,HF_EG,0.268002,-0.704897,-88705.152361,1.249542e+07,1.258412e+07,-8.984931,2.694806e-19,0.989502,9.243180e+04,9.433680e+06,9.341248e+06,-5.461298,4.773979e-08,0.742304,0.754971,3.785304,3.758622,analytical_profit_pre_vs_analytical_profit_post
1,TB_IQ,3.349913,2.162191,69286.517316,3.273745e+06,3.204458e+06,-2.983866,2.851123e-03,1.176260,4.176147e+04,3.592122e+06,3.550361e+06,-1.986907,4.696586e-02,1.107944,1.097252,3.107290,3.174476,analytical_profit_pre_vs_analytical_profit_post
2,TB_QA,-3.301092,-0.421764,-15618.145411,3.687434e+06,3.703052e+06,-0.136515,8.914162e-01,-2.810264,-2.198198e+05,7.602215e+06,7.822035e+06,0.176862,8.596215e-01,2.112321,2.061655,4.650328,4.630715,analytical_profit_pre_vs_analytical_profit_post
3,TB_AE,-0.449917,-1.002343,-144301.659261,1.425213e+07,1.439643e+07,-3.080046,2.071090e-03,0.561151,3.031768e+05,5.433086e+07,5.402768e+07,-3.619941,2.952399e-04,3.752851,3.812121,5.347960,5.294355,analytical_profit_pre_vs_analytical_profit_post
4,TB_OM,9.767336,3.324830,90400.473930,2.809351e+06,2.718950e+06,-0.469791,6.385146e-01,6.649826,1.762249e+05,2.826293e+06,2.650068e+06,-1.001636,3.165566e-01,0.974666,1.006031,4.074902,4.210386,analytical_profit_pre_vs_analytical_profit_post
5,GV_MD,-2.243877,3.141377,6820.808092,2.239488e+05,2.171279e+05,0.786639,4.316314e-01,-4.962150,-2.546412e+04,4.877029e+05,5.131670e+05,-0.299084,7.649778e-01,2.363431,2.177743,1.971417,2.033347,analytical_profit_pre_vs_analytical_profit_post
6,GV_RO,-3.562108,-0.954023,-31059.976005,3.224625e+06,3.255685e+06,-1.699684,8.920700e-02,-2.565648,-2.658113e+05,1.009458e+07,1.036039e+07,-1.647752,9.944038e-02,3.182247,3.130467,2.178872,2.158085,analytical_profit_pre_vs_analytical_profit_post
7,TB_KW,7.668348,4.576163,305850.893934,6.989417e+06,6.683566e+06,-2.651003,8.031673e-03,3.046968,4.351360e+05,1.471609e+07,1.428095e+07,-3.123895,1.788322e-03,2.136726,2.105481,5.458802,5.708605,analytical_profit_pre_vs_analytical_profit_post
8,HS_SA,-1.055169,-2.804712,-678816.969159,2.352392e+07,2.420274e+07,-4.750941,2.028695e-06,1.833023,1.144456e+06,6.357991e+07,6.243546e+07,1.232326,2.178368e-01,2.579686,2.702777,1.650270,1.603985,analytical_profit_pre_vs_analytical_profit_post
9,TB_BH,2.795417,1.408497,39964.053819,2.877318e+06,2.837354e+06,-2.017075,4.372538e-02,1.386621,8.342761e+04,6.100042e+06,6.016614e+06,0.266844,7.896012e-01,2.120501,2.120044,6.022854,6.107685,analytical_profit_pre_vs_analytical_profit_post


In [52]:
sg_df[sg_df['entity_id'] == 'HS_SA']

,entity_id,customer_id,is_customer_holdout,orders_pre,orders_post,analytical_profit_pre,analytical_profit_post
7,HS_SA,1735322500270.7634562015.tdcdxsphov,False,<NA>,<NA>,NaN,NaN
11,HS_SA,11270239,False,<NA>,<NA>,NaN,NaN
17,HS_SA,17067113,False,<NA>,<NA>,NaN,NaN
24,HS_SA,20813887,False,<NA>,<NA>,NaN,NaN
27,HS_SA,17204672,False,<NA>,<NA>,NaN,NaN
...,...,...,...,...,...,...,...
93887474,HS_SA,1738143275769.3799108385.ppcikjxlyd,False,<NA>,<NA>,NaN,NaN
93887476,HS_SA,1738992463600.6852489086.qqohwvrlsg,False,<NA>,<NA>,NaN,NaN
93887480,HS_SA,1717936772156.8931045486.yapcuoklha,False,<NA>,<NA>,NaN,NaN
93887488,HS_SA,19143288,False,<NA>,<NA>,NaN,NaN


In [21]:
mkt_df = mkt_df[['global_entity_id','non_holdout_dh_funded_basket_incentives','holdout_dh_funded_basket_incentives','holdout_delivery_fee_net','non_holdout_delivery_fee_net'
        ,'holdout_customers','non_holdout_customers','flgp_holdout','flgp_non_holdout','holdout_cost_of_sales','non_holdout_cost_of_sales'
        ,'holdout_delivery_fee_net','non_holdout_delivery_fee_net']]

In [22]:
mkt_df.to_csv('raw_data.csv', index=False) 

In [5]:
mkt_df.dtypes

analysis_week                                   dbdate
global_entity_id                                object
release_date                                    dbdate
holdout_customers                                Int64
non_holdout_customers                            Int64
                                                ...   
non_holdout_subscription_fee_net               float64
holdout_subscription_vendor_subsidy_net        float64
non_holdout_subscription_vendor_subsidy_net    float64
holdout_other_revenue_net                      float64
non_holdout_other_revenue_net                  float64
Length: 72, dtype: object

In [6]:
mkt_df['discount_correction'] = mkt_df['non_holdout_dh_funded_basket_incentives'] / mkt_df['holdout_dh_funded_basket_incentives']

mkt_df['non_holdout_discount_o_df'] = mkt_df['non_holdout_dh_funded_basket_incentives'] / mkt_df['non_holdout_delivery_fee_net']

mkt_df['holdout_discount_o_df'] = mkt_df['holdout_dh_funded_basket_incentives'] / mkt_df['holdout_delivery_fee_net']

mkt_df['discount_correction_holdout'] = mkt_df['non_holdout_discount_o_df'] / mkt_df['holdout_discount_o_df']

mkt_df['holdout_dh_funded_basket_incentives_corrected'] = mkt_df['discount_correction_holdout'] * mkt_df['holdout_dh_funded_basket_incentives']

mkt_df[['holdout_dh_funded_basket_incentives_corrected','holdout_dh_funded_basket_incentives']]

mkt_df['flgp_holdout_corrected'] = mkt_df['flgp_holdout'] + mkt_df['holdout_dh_funded_basket_incentives_corrected']



In [10]:
mkt_df['scaled_holdout_total_flgp_cuped'] = (mkt_df['flgp_holdout_corrected'] / mkt_df['holdout_customers']) * mkt_df['non_holdout_customers']
mkt_df['scaled_holdout_total_orders_cuped'] = (mkt_df['holdout_orders'] / mkt_df['holdout_customers']) * mkt_df['non_holdout_customers']

mkt_df['holdout_flgp_per_order_cuped'] = np.where(
    mkt_df['holdout_orders'] != 0, 
    mkt_df['flgp_holdout_corrected'] / mkt_df['holdout_orders'], 
    np.nan
)

mkt_df['non_holdout_flgp_per_order_cuped'] = np.where(
    mkt_df['non_holdout_orders'] != 0, 
    mkt_df['flgp_non_holdout'] / mkt_df['non_holdout_orders'], 
    np.nan
)

mkt_df['holdout_orders_per_user_cuped'] = np.where(
    mkt_df['holdout_customers'] != 0, 
    mkt_df['holdout_orders'] / mkt_df['holdout_customers'], 
    np.nan
)

mkt_df['non_holdout_orders_per_user_cuped'] = np.where(
    mkt_df['non_holdout_customers'] != 0, 
    mkt_df['non_holdout_orders'] / mkt_df['non_holdout_customers'], 
    np.nan
)

mkt_df['incremental_orders_cuped'] = mkt_df['non_holdout_orders'] - mkt_df['scaled_holdout_total_orders_cuped']
mkt_df['incremental_flgp_cuped'] = mkt_df['flgp_non_holdout'] - mkt_df['scaled_holdout_total_flgp_cuped']

mkt_df['sustainable_growth'] = np.where(
    mkt_df['scaled_holdout_total_orders_cuped'] != 0, 
    ((mkt_df['incremental_orders_cuped'] + 
      (mkt_df['incremental_flgp_cuped'] / mkt_df['non_holdout_flgp_per_order_cuped'])) 
     / mkt_df['scaled_holdout_total_orders_cuped']) * 100, 
    np.nan
)

In [12]:
mkt_df[mkt_df['global_entity_id'] == 'PO_FI']['non_holdout_dh_funded_basket_incentives'] / mkt_df[mkt_df['global_entity_id'] == 'PO_FI']['non_holdout_delivery_fee_net']

23   -0.329079
dtype: float64

In [13]:
mkt_df[mkt_df['global_entity_id'] == 'PO_FI']['holdout_dh_funded_basket_incentives_corrected'] / mkt_df[mkt_df['global_entity_id'] == 'PO_FI']['holdout_delivery_fee_net']

23   -0.329079
dtype: float64

In [ ]:
# Normalize for Population Differences
holdout_user_count = data['is_customer_holdout'].sum()
non_holdout_user_count = (~data['is_customer_holdout']).sum()

scaled_holdout_total_flgp_cuped = (holdout_total_flgp_cuped / holdout_user_count) * non_holdout_user_count if holdout_user_count != 0 else np.nan
scaled_holdout_total_orders_cuped = (holdout_total_orders_cuped / holdout_user_count) * non_holdout_user_count if holdout_user_count != 0 else np.nan

# FLGP per Order and Orders per User (CUPED)
holdout_flgp_per_order_cuped = holdout_total_flgp_cuped / holdout_total_orders_cuped if holdout_total_orders_cuped != 0 else np.nan
non_holdout_flgp_per_order_cuped = non_holdout_total_flgp_cuped / non_holdout_total_orders_cuped if non_holdout_total_orders_cuped != 0 else np.nan

holdout_orders_per_user_cuped = holdout_total_orders_cuped / holdout_user_count if holdout_user_count != 0 else np.nan
non_holdout_orders_per_user_cuped = non_holdout_total_orders_cuped / non_holdout_user_count if non_holdout_user_count != 0 else np.nan

# Incremental Differences (CUPED)
incremental_orders_cuped = non_holdout_total_orders_cuped - scaled_holdout_total_orders_cuped
incremental_flgp_cuped = non_holdout_total_flgp_cuped - scaled_holdout_total_flgp_cuped

# Percentage Changes (CUPED)
percentage_change_orders_cuped = ((incremental_orders_cuped) / abs(scaled_holdout_total_orders_cuped)) * 100 if scaled_holdout_total_orders_cuped != 0 else np.nan
percentage_change_flgp_cuped = ((incremental_flgp_cuped) / abs(scaled_holdout_total_flgp_cuped)) * 100 if scaled_holdout_total_flgp_cuped != 0 else np.nan

# Sustainable Growth Calculation
sustainable_growth = ((incremental_orders_cuped + (incremental_flgp_cuped / non_holdout_flgp_per_order_cuped)) / scaled_holdout_total_orders_cuped) * 100 if scaled_holdout_total_orders_cuped != 0 else np.nan

In [ ]:
    sustainable_growth = ((incremental_orders_cuped + (incremental_flgp_cuped / non_holdout_flgp_per_order_cuped)) / scaled_holdout_total_orders_cuped) * 100 if scaled_holdout_total_orders_cuped != 0 else np.nan


In [27]:
mkt_df['holdout_delivery_fee_per_order'] = mkt_df['holdout_delivery_fee_net'] / mkt_df['holdout_orders'] 
mkt_df['non_holdout_delivery_fee_per_order'] = mkt_df['non_holdout_delivery_fee_net'] / mkt_df['non_holdout_orders'] 

mkt_df['holdout_discount_per_order'] = mkt_df['holdout_dh_funded_basket_incentives'] / mkt_df['holdout_orders'] 
mkt_df['non_holdout_discount_per_order'] = mkt_df['non_holdout_dh_funded_basket_incentives'] / mkt_df['non_holdout_orders'] 

mkt_df['holdout_revenue_per_order'] = mkt_df['holdout_revenue_net'] / mkt_df['holdout_orders'] 
mkt_df['non_holdout_revenue_per_order'] = mkt_df['non_holdout_revenue_net'] / mkt_df['non_holdout_orders'] 

mkt_df['holdout_cos_per_order'] = mkt_df['holdout_cost_of_sales'] / mkt_df['holdout_orders'] 
mkt_df['non_holdout_cos_per_order'] = mkt_df['non_holdout_cost_of_sales'] / mkt_df['non_holdout_orders'] 

mkt_df['holdout_delivery_costs_per_order'] = mkt_df['holdout_delivery_costs'] / mkt_df['holdout_orders'] 
mkt_df['non_holdout_delivery_costs_per_order'] = mkt_df['non_holdout_delivery_costs'] / mkt_df['non_holdout_orders'] 

mkt_df['holdout_cost_of_goods_sold_per_order'] = mkt_df['holdout_cost_of_goods_sold'] / mkt_df['holdout_orders'] 
mkt_df['non_holdout_holdout_cost_of_goods_sold_per_order'] = mkt_df['non_holdout_cost_of_goods_sold'] / mkt_df['non_holdout_orders'] 

mkt_df['holdout_payment_costs_per_order'] = mkt_df['holdout_payment_costs'] / mkt_df['holdout_orders'] 
mkt_df['non_holdout_payment_costs_per_order'] = mkt_df['non_holdout_payment_costs'] / mkt_df['non_holdout_orders']

mkt_df['holdout_recooks_and_wastage_per_order'] = mkt_df['holdout_recooks_and_wastage'] / mkt_df['holdout_orders'] 
mkt_df['non_holdout_recooks_and_wastage_per_order'] = mkt_df['non_holdout_recooks_and_wastage'] / mkt_df['non_holdout_orders']

mkt_df['holdout_refunds_and_compensation_per_order'] = mkt_df['holdout_refunds_and_compensation'] / mkt_df['holdout_orders'] 
mkt_df['non_holdout_refunds_and_compensation_per_order'] = mkt_df['non_holdout_refunds_and_compensation'] / mkt_df['non_holdout_orders']

mkt_df['holdout_semi_variable_delivery_costs_per_order'] = mkt_df['holdout_semi_variable_delivery_costs'] / mkt_df['holdout_orders'] 
mkt_df['non_holdout_semi_variable_delivery_costs_per_order'] = mkt_df['non_holdout_semi_variable_delivery_costs'] / mkt_df['non_holdout_orders']

mkt_df['holdout_contact_center_costs_per_order'] = mkt_df['holdout_contact_center_costs'] / mkt_df['holdout_orders'] 
mkt_df['non_holdout_contact_center_costs_per_order'] = mkt_df['non_holdout_contact_center_costs'] / mkt_df['non_holdout_orders']

mkt_df['holdout_other_non_specific_delivery_costs_per_order'] = mkt_df['holdout_other_non_specific_delivery_costs'] / mkt_df['holdout_orders'] 
mkt_df['non_holdout_other_non_specific_delivery_costs_per_order'] = mkt_df['non_holdout_other_non_specific_delivery_costs'] / mkt_df['non_holdout_orders']

mkt_df['holdout_commission_fee_net_per_order'] = mkt_df['holdout_commission_fee_net'] / mkt_df['holdout_orders'] 
mkt_df['non_holdout_commission_fee_net_per_order'] = mkt_df['non_holdout_commission_fee_net'] / mkt_df['non_holdout_orders']

mkt_df['holdout_sale_of_goods_revenue_net_per_order'] = mkt_df['holdout_sale_of_goods_revenue_net'] / mkt_df['holdout_orders'] 
mkt_df['non_holdout_sale_of_goods_revenue_net_per_order'] = mkt_df['non_holdout_sale_of_goods_revenue_net'] / mkt_df['non_holdout_orders']

mkt_df['holdout_card_fee_net_per_order'] = mkt_df['holdout_card_fee_net'] / mkt_df['holdout_orders'] 
mkt_df['non_holdout_card_fee_net_per_order'] = mkt_df['non_holdout_card_fee_net'] / mkt_df['non_holdout_orders']

mkt_df['holdout_priority_fee_net_per_order'] = mkt_df['holdout_priority_fee_net'] / mkt_df['holdout_orders'] 
mkt_df['non_holdout_priority_fee_net_per_order'] = mkt_df['non_holdout_priority_fee_net'] / mkt_df['non_holdout_orders']

mkt_df['holdout_bad_weather_fee_net_per_order'] = mkt_df['holdout_bad_weather_fee_net'] / mkt_df['holdout_orders'] 
mkt_df['non_holdout_bad_weather_fee_net_per_order'] = mkt_df['non_holdout_bad_weather_fee_net'] / mkt_df['non_holdout_orders']

mkt_df['holdout_vendor_funded_delivery_fee_subsidy_net_per_order'] = mkt_df['holdout_vendor_funded_delivery_fee_subsidy_net'] / mkt_df['holdout_orders'] 
mkt_df['non_holdout_vendor_funded_delivery_fee_subsidy_net_per_order'] = mkt_df['non_holdout_vendor_funded_delivery_fee_subsidy_net'] / mkt_df['non_holdout_orders']

mkt_df['holdout_service_fee_net_per_order'] = mkt_df['holdout_service_fee_net'] / mkt_df['holdout_orders'] 
mkt_df['non_holdout_service_fee_net_per_order'] = mkt_df['non_holdout_service_fee_net'] / mkt_df['non_holdout_orders']

mkt_df['holdout_listing_fee_net_per_order'] = mkt_df['holdout_listing_fee_net'] / mkt_df['holdout_orders'] 
mkt_df['non_holdout_listing_fee_net_per_order'] = mkt_df['non_holdout_listing_fee_net'] / mkt_df['non_holdout_orders']

mkt_df['holdout_sublease_revenue_net_per_order'] = mkt_df['holdout_sublease_revenue_net'] / mkt_df['holdout_orders'] 
mkt_df['non_holdout_sublease_revenue_net_per_order'] = mkt_df['non_holdout_sublease_revenue_net'] / mkt_df['non_holdout_orders']

mkt_df['holdout_ads_revenue_net_per_order'] = mkt_df['holdout_ads_revenue_net'] / mkt_df['holdout_orders'] 
mkt_df['non_holdout_ads_revenue_net_per_order'] = mkt_df['non_holdout_ads_revenue_net'] / mkt_df['non_holdout_orders']

mkt_df['holdout_joker_vendor_fee_net_per_order'] = mkt_df['holdout_joker_vendor_fee_net'] / mkt_df['holdout_orders'] 
mkt_df['non_holdout_joker_vendor_fee_net_per_order'] = mkt_df['non_holdout_joker_vendor_fee_net'] / mkt_df['non_holdout_orders']

mkt_df['holdout_other_ncr_net_per_order'] = mkt_df['holdout_other_ncr_net'] / mkt_df['holdout_orders'] 
mkt_df['non_holdout_other_ncr_net_per_order'] = mkt_df['non_holdout_other_ncr_net'] / mkt_df['non_holdout_orders']

mkt_df['holdout_small_basket_fee_net_per_order'] = mkt_df['holdout_small_basket_fee_net'] / mkt_df['holdout_orders'] 
mkt_df['non_holdout_small_basket_fee_net_per_order'] = mkt_df['non_holdout_small_basket_fee_net'] / mkt_df['non_holdout_orders']

mkt_df['holdout_subscription_fee_net_per_order'] = mkt_df['holdout_subscription_fee_net'] / mkt_df['holdout_orders'] 
mkt_df['non_holdout_subscription_fee_net_per_order'] = mkt_df['non_holdout_subscription_fee_net'] / mkt_df['non_holdout_orders']

mkt_df['holdout_other_revenue_net_per_order'] = mkt_df['holdout_other_revenue_net'] / mkt_df['holdout_orders'] 
mkt_df['non_holdout_other_revenue_net_per_order'] = mkt_df['non_holdout_other_revenue_net'] / mkt_df['non_holdout_orders']

# tmp_cos = mkt_df[['global_entity_id','holdout_cost_of_goods_sold_per_order','non_holdout_holdout_cost_of_goods_sold_per_order',
# 'holdout_recooks_and_wastage_per_order','non_holdout_recooks_and_wastage_per_order',
# 'holdout_refunds_and_compensation_per_order','non_holdout_refunds_and_compensation_per_order',
# 'holdout_semi_variable_delivery_costs_per_order','non_holdout_semi_variable_delivery_costs_per_order',
# 'holdout_contact_center_costs_per_order','non_holdout_contact_center_costs_per_order',
# 'holdout_other_non_specific_delivery_costs_per_order','non_holdout_other_non_specific_delivery_costs_per_order',
# 'holdout_delivery_costs_per_order','non_holdout_delivery_costs_per_order','holdout_payment_costs_per_order',
# 'non_holdout_payment_costs_per_order', 'holdout_cos_per_order','non_holdout_cos_per_order', 'sustainable_growth', 
# ]]

# tmp_cos['sustainable_growth'] = 100 * tmp_cos['sustainable_growth']

# tmp_cos.sort_values(by = 'sustainable_growth').transpose()

# tmp_revenue = mkt_df[['global_entity_id',
# 'holdout_commission_fee_net_per_order', 'non_holdout_commission_fee_net_per_order',
# 'holdout_sale_of_goods_revenue_net_per_order','non_holdout_sale_of_goods_revenue_net_per_order',
# 'holdout_card_fee_net_per_order','non_holdout_card_fee_net_per_order',
# 'holdout_delivery_fee_per_order','non_holdout_delivery_fee_per_order',
# 'holdout_priority_fee_net_per_order','non_holdout_priority_fee_net_per_order',
# 'holdout_bad_weather_fee_net_per_order','non_holdout_bad_weather_fee_net_per_order',
# 'holdout_vendor_funded_delivery_fee_subsidy_net_per_order','non_holdout_vendor_funded_delivery_fee_subsidy_net_per_order',
# 'holdout_service_fee_net_per_order','non_holdout_service_fee_net_per_order',
# 'holdout_listing_fee_net_per_order','non_holdout_listing_fee_net_per_order',
# 'holdout_sublease_revenue_net_per_order','non_holdout_sublease_revenue_net_per_order',
# 'holdout_ads_revenue_net_per_order','non_holdout_ads_revenue_net_per_order',
# 'holdout_joker_vendor_fee_net_per_order','non_holdout_joker_vendor_fee_net_per_order',
# 'holdout_other_ncr_net_per_order','non_holdout_other_ncr_net_per_order',
# 'holdout_small_basket_fee_net_per_order','non_holdout_small_basket_fee_net_per_order',
# 'holdout_subscription_fee_net_per_order','non_holdout_subscription_fee_net_per_order',
# 'holdout_other_revenue_net_per_order','non_holdout_other_revenue_net_per_order',
# 'holdout_revenue_per_order','non_holdout_revenue_per_order',
# 'sustainable_growth'
# ]]

# tmp_revenue['sustainable_growth'] = 100 * tmp_revenue['sustainable_growth']

# tmp_revenue.sort_values(by = 'sustainable_growth').transpose()


tmp_combined = mkt_df[['global_entity_id',
'holdout_discount_per_order', 'non_holdout_discount_per_order',
'holdout_cos_per_order', 'non_holdout_cos_per_order',
'holdout_revenue_per_order', 'non_holdout_revenue_per_order',
'flgpo_holdout', 'flgpo_non_holdout',
'flgp_growth', 'orders_growth',
'sustainable_growth'
]]

tmp_combined['discount_diff'] = 100 * ((tmp_combined['non_holdout_discount_per_order'] - tmp_combined['holdout_discount_per_order']) / tmp_combined['holdout_discount_per_order'])
tmp_combined['cos_diff'] = 100 * ((tmp_combined['non_holdout_cos_per_order'] - tmp_combined['holdout_cos_per_order']) / tmp_combined['holdout_cos_per_order'] )
tmp_combined['revenue_diff'] = 100 * ((tmp_combined['non_holdout_revenue_per_order'] - tmp_combined['holdout_revenue_per_order']) / tmp_combined['holdout_revenue_per_order'] )


tmp_combined['sustainable_growth'] = 100 * tmp_combined['sustainable_growth']

a = tmp_combined.sort_values(by = 'sustainable_growth').transpose()



/var/folders/7c/hjrbzbpn61jgnsn14y5f3l7c0000gq/T/ipykernel_69416/3446076556.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_combined['discount_diff'] = 100 * ((tmp_combined['non_holdout_discount_per_order'] - tmp_combined['holdout_discount_per_order']) / tmp_combined['holdout_discount_per_order'])
/var/folders/7c/hjrbzbpn61jgnsn14y5f3l7c0000gq/T/ipykernel_69416/3446076556.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_combined['cos_diff'] = 100 * ((tmp_combined['non_holdout_cos_per_orde

In [32]:
tmp_combined.to_csv('summary.csv', index=False) 

In [31]:
tmp_combined

,global_entity_id,holdout_discount_per_order,non_holdout_discount_per_order,holdout_cos_per_order,non_holdout_cos_per_order,holdout_revenue_per_order,non_holdout_revenue_per_order,flgpo_holdout,flgpo_non_holdout,flgp_growth,orders_growth,sustainable_growth,discount_diff,cos_diff,revenue_diff
0,OP_SE,-0.562563,-0.569406,-7.535926,-7.503377,9.665415,9.677038,1.566925,1.604255,0.040536,0.016324,5.591638,1.216289,-0.431922,0.120249
1,PY_CR,-0.160452,-0.1812,-3.303174,-3.296414,4.160552,4.157648,0.696926,0.680034,-0.009733,0.014865,0.489048,12.930979,-0.20465,-0.069787
2,PY_CL,-0.333287,-0.336183,-2.337267,-2.328908,4.542692,4.594517,1.872138,1.929425,0.059669,0.028206,8.610262,0.869144,-0.357653,1.140838
3,PY_EC,-0.104739,-0.106196,-1.975063,-1.978166,2.985279,2.95514,0.905477,0.870778,-0.025377,0.013461,-1.292698,1.39145,0.157136,-1.009571
4,PY_BO,-0.121832,-0.123893,-1.658028,-1.653136,2.781008,2.754924,1.001148,0.977896,0.008760,0.032746,4.171442,1.69134,-0.29504,-0.937932
5,PY_GT,-0.133048,-0.145103,-2.368701,-2.369064,3.547255,3.547527,1.045505,1.033360,-0.009630,0.002010,-0.773289,9.060712,0.015295,0.007668
6,AP_PA,-0.0454,-0.044698,-2.663909,-2.647166,4.475635,4.382102,1.766326,1.690238,-0.011136,0.033379,2.174113,-1.545089,-0.628497,-2.089819
7,FP_BD,-0.122221,-0.122566,-0.358997,-0.357876,1.38195,1.363143,0.900731,0.882701,0.018395,0.039196,5.796682,0.282216,-0.312242,-1.360846
8,PY_AR,-0.127941,-0.129386,-1.903146,-1.902154,4.212093,4.212593,2.181006,2.181053,0.015539,0.015517,3.105534,1.129525,-0.052121,0.011885
9,PY_SV,-0.109984,-0.106042,-2.034018,-2.041232,3.48963,3.51172,1.345627,1.364446,0.019626,0.005563,2.491920,-3.584075,0.354622,0.63302


In [ ]:
mkt_df['delio']

In [495]:
mkt_df[mkt_df['global_entity_id'] == 'PO_FI']['holdout_dh_funded_basket_incentives'] / mkt_df[mkt_df['global_entity_id'] == 'PO_FI']['holdout_delivery_fee_net']

6   -0.244227
dtype: float64

In [496]:
mkt_df[mkt_df['global_entity_id'] == 'PO_FI']['non_holdout_dh_funded_basket_incentives'] / mkt_df[mkt_df['global_entity_id'] == 'PO_FI']['non_holdout_delivery_fee_net']

6   -0.329079
dtype: float64

In [499]:
((-0.329079 / -0.244227) * mkt_df[mkt_df['global_entity_id'] == 'PO_FI']['holdout_dh_funded_basket_incentives']) / mkt_df[mkt_df['global_entity_id'] == 'PO_FI']['holdout_delivery_fee_net']


6   -0.329078
dtype: float64

In [466]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind

def calculate_sustainable_growth(data, pre_profit, post_profit):
    
    data = data.copy()

    entity = data['entity_id'].iloc[0]

    # Apply CUPED to FLGP
    data_flgp = data.dropna(subset=[pre_profit, post_profit])
    theta_flgp = np.cov(data_flgp[pre_profit], data_flgp[post_profit])[0, 1] / np.var(data_flgp[pre_profit])
    data_flgp['flgp_post_cuped'] = data_flgp[post_profit] - theta_flgp * (data_flgp[pre_profit] - data_flgp[pre_profit].mean())

    # Apply CUPED to Orders
    data_orders = data.dropna(subset=['orders_pre', 'orders_post'])
    theta_orders = np.cov(data_orders['orders_pre'], data_orders['orders_post'])[0, 1] / np.var(data_orders['orders_pre'])
    data_orders['orders_post_cuped'] = data_orders['orders_post'] - theta_orders * (data_orders['orders_pre'] - data_orders['orders_pre'].mean())

    # Total FLGP and Orders (CUPED)
    holdout_total_flgp_cuped = data_flgp.loc[data_flgp['is_customer_holdout'], 'flgp_post_cuped'].sum()
    non_holdout_total_flgp_cuped = data_flgp.loc[~data_flgp['is_customer_holdout'], 'flgp_post_cuped'].sum()

    holdout_total_orders_cuped = data_orders.loc[data_orders['is_customer_holdout'], 'orders_post_cuped'].sum()
    non_holdout_total_orders_cuped = data_orders.loc[~data_orders['is_customer_holdout'], 'orders_post_cuped'].sum()

    # Normalize for Population Differences
    holdout_user_count = data['is_customer_holdout'].sum()
    non_holdout_user_count = (~data['is_customer_holdout']).sum()

    scaled_holdout_total_flgp_cuped = (holdout_total_flgp_cuped / holdout_user_count) * non_holdout_user_count if holdout_user_count != 0 else np.nan
    scaled_holdout_total_orders_cuped = (holdout_total_orders_cuped / holdout_user_count) * non_holdout_user_count if holdout_user_count != 0 else np.nan

    # FLGP per Order and Orders per User (CUPED)
    holdout_flgp_per_order_cuped = holdout_total_flgp_cuped / holdout_total_orders_cuped if holdout_total_orders_cuped != 0 else np.nan
    non_holdout_flgp_per_order_cuped = non_holdout_total_flgp_cuped / non_holdout_total_orders_cuped if non_holdout_total_orders_cuped != 0 else np.nan

    holdout_orders_per_user_cuped = holdout_total_orders_cuped / holdout_user_count if holdout_user_count != 0 else np.nan
    non_holdout_orders_per_user_cuped = non_holdout_total_orders_cuped / non_holdout_user_count if non_holdout_user_count != 0 else np.nan

    # Incremental Differences (CUPED)
    incremental_orders_cuped = non_holdout_total_orders_cuped - scaled_holdout_total_orders_cuped
    incremental_flgp_cuped = non_holdout_total_flgp_cuped - scaled_holdout_total_flgp_cuped

    # Percentage Changes (CUPED)
    percentage_change_orders_cuped = ((incremental_orders_cuped) / abs(scaled_holdout_total_orders_cuped)) * 100 if scaled_holdout_total_orders_cuped != 0 else np.nan
    percentage_change_flgp_cuped = ((incremental_flgp_cuped) / abs(scaled_holdout_total_flgp_cuped)) * 100 if scaled_holdout_total_flgp_cuped != 0 else np.nan

    # Sustainable Growth Calculation
    sustainable_growth = ((incremental_orders_cuped + (incremental_flgp_cuped / non_holdout_flgp_per_order_cuped)) / scaled_holdout_total_orders_cuped) * 100 if scaled_holdout_total_orders_cuped != 0 else np.nan

    # T-tests for significance
    t_stat_orders, p_value_orders = ttest_ind(
        data_orders.loc[data_orders['is_customer_holdout'], 'orders_post_cuped'],
        data_orders.loc[~data_orders['is_customer_holdout'], 'orders_post_cuped'],
        equal_var=False
    )

    t_stat_flgp, p_value_flgp = ttest_ind(
        data_flgp.loc[data_flgp['is_customer_holdout'], 'flgp_post_cuped'],
        data_flgp.loc[~data_flgp['is_customer_holdout'], 'flgp_post_cuped'],
        equal_var=False
    )

    return {
        'entity': entity,
        'sustainable_growth': sustainable_growth,
        'percentage_change_orders_cuped': percentage_change_orders_cuped,
        'incremental_orders_cuped': incremental_orders_cuped,
        'non_holdout_total_orders_cuped':non_holdout_total_orders_cuped,
        'holdout_total_orders_cuped':scaled_holdout_total_orders_cuped,
        't_stat_orders': t_stat_orders,
        'p_value_orders': p_value_orders,
        'percentage_change_flgp_cuped': percentage_change_flgp_cuped,
        'incremental_flgp_cuped': incremental_flgp_cuped,
        'non_holdout_total_flgp_cuped': non_holdout_total_flgp_cuped,
        'holdout_total_flgp_cuped':scaled_holdout_total_flgp_cuped,
        't_stat_flgp': t_stat_flgp,
        'p_value_flgp': p_value_flgp,
        'holdout_flgp_per_order_cuped': holdout_flgp_per_order_cuped,
        'non_holdout_flgp_per_order_cuped': non_holdout_flgp_per_order_cuped,
        'holdout_orders_per_user_cuped': holdout_orders_per_user_cuped,
        'non_holdout_orders_per_user_cuped': non_holdout_orders_per_user_cuped
    }


# results = []
# for entity in mkt_df['entity_id'].unique():
#     entity_data = mkt_df[mkt_df['entity_id'] == entity]
#     result = calculate_sustainable_growth(entity_data, 'local_analytical_profit_pre','local_analytical_profit_post')
#     result['entity_id'] = entity
#     results.append(result)

# pd.DataFrame(results)

mkt_df['profit_pre'] = mkt_df['revenue_pre'] + mkt_df['cost_of_sales_pre']
mkt_df['profit_post'] = mkt_df['revenue_pre'] + mkt_df['cost_of_sales_pre']

# Define the list of pre and post metrics
metric_pairs = [
    ('analytical_profit_pre', 'analytical_profit_post'),
    ('local_analytical_profit_pre', 'local_analytical_profit_post'),
    ('profit_pre', 'profit_post')
]

results = []

# Loop through each metric pair
for pre_metric, post_metric in metric_pairs:
    for entity in mkt_df['entity_id'].unique():
        entity_data = mkt_df[mkt_df['entity_id'] == entity]
        result = calculate_sustainable_growth(entity_data, pre_metric, post_metric)
        
        # Add entity_id and metric information to the result
        result['metric_used'] = f"{pre_metric}_vs_{post_metric}"
        
        results.append(result)

# Convert results to DataFrame
final_results_df = pd.DataFrame(results)


final_results_df = final_results_df.dropna(subset= 'sustainable_growth')

final_results_df.to_csv('profitable_growth.csv', index=False) 

/var/folders/7c/hjrbzbpn61jgnsn14y5f3l7c0000gq/T/ipykernel_92164/300001130.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_flgp['flgp_post_cuped'] = data_flgp[post_profit] - theta_flgp * (data_flgp[pre_profit] - data_flgp[pre_profit].mean())
/var/folders/7c/hjrbzbpn61jgnsn14y5f3l7c0000gq/T/ipykernel_92164/300001130.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_flgp['flgp_post_cuped'] = data_flgp[post_profit] - theta_flgp * (data_flgp[pre_profit] - data_flgp[pre_profit].mean())
/var/fol

,entity,sustainable_growth,percentage_change_orders_cuped,incremental_orders_cuped,non_holdout_total_orders_cuped,holdout_total_orders_cuped,t_stat_orders,p_value_orders,percentage_change_flgp_cuped,incremental_flgp_cuped,non_holdout_total_flgp_cuped,holdout_total_flgp_cuped,t_stat_flgp,p_value_flgp,holdout_flgp_per_order_cuped,non_holdout_flgp_per_order_cuped,holdout_orders_per_user_cuped,non_holdout_orders_per_user_cuped,entity_id
0,EF_GR,7.729787,6.110630,138111.699112,2.398299e+06,2.260188e+06,-8.072846,7.224959e-16,1.549559,33514.267634,2.196340e+06,2.162826e+06,-0.441929,6.585492e-01,0.956923,0.915791,2.178339,2.311449,EF_GR
1,FP_TW,1.835712,0.844197,78427.237009,9.368589e+06,9.290161e+06,-2.193176,2.829853e-02,0.992978,62693.155414,6.376345e+06,6.313652e+06,-1.534955,1.248041e-01,0.679606,0.680609,3.269494,3.297095,FP_TW
2,YS_TR,-32.182999,1.723559,143208.367893,8.452083e+06,8.308874e+06,-4.254027,2.101781e-05,24.999284,514917.054223,-1.544810e+06,-2.059727e+06,-7.681676,1.603083e-14,-0.247895,-0.182773,1.985008,2.019221,YS_TR
3,PY_AR,1.532740,0.787178,61342.206187,7.854017e+06,7.792675e+06,-2.037969,4.155684e-02,0.745252,102134.123058,1.380677e+07,1.370464e+07,-2.840333,4.509081e-03,1.758657,1.757925,2.380435,2.399173,PY_AR
4,MJM_AT,3.401775,-0.075154,-707.678628,9.409250e+05,9.416327e+05,0.071441,9.430481e-01,3.604981,118602.074819,3.408552e+06,3.289950e+06,-1.746552,8.077751e-02,3.493878,3.622554,2.288688,2.286968,MJM_AT
5,FP_MY,2.536091,-0.468499,-24918.010045,5.293769e+06,5.318687e+06,0.958192,3.379700e-01,3.112697,101758.162683,3.370890e+06,3.269132e+06,-3.738859,1.852270e-04,0.614650,0.636766,1.902500,1.893587,FP_MY
6,PY_EC,-1.420451,1.862626,18538.614435,1.013833e+06,9.952947e+05,-1.586408,1.126790e-01,-3.122407,-27440.173565,8.513746e+05,8.788148e+05,1.642152,1.006245e-01,0.882969,0.839758,2.125594,2.165186,PY_EC
7,DJ_CZ,-0.983975,0.144272,1662.130284,1.153745e+06,1.152083e+06,-0.146085,8.838573e-01,-1.114070,-13753.049398,1.220734e+06,1.234487e+06,0.694165,4.876073e-01,1.071526,1.058062,2.250641,2.253888,DJ_CZ
8,NP_HU,-0.438168,-0.027398,-306.158430,1.117159e+06,1.117465e+06,0.026504,9.788557e-01,-0.409202,-3593.909816,8.746799e+05,8.782738e+05,-0.102614,9.182730e-01,0.785952,0.782950,2.397249,2.396592,NP_HU
9,FP_MM,0.771945,1.482770,10888.645986,7.452337e+05,7.343451e+05,-1.007453,3.137495e-01,-0.695567,-3391.743385,4.842313e+05,4.876230e+05,-0.098895,9.212277e-01,0.664024,0.649771,2.016146,2.046041,FP_MM
